<a href="https://colab.research.google.com/github/gian3846/CGS616/blob/main/Code_for_AR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AR-1


## Stack-Plan Generator

**Purpose:**
Assign “blocks” (spaces) to building floors according to different stacking modes (“centralized”, “semi”, “decentralized”), respecting area and capacity constraints and generating summary reports.

---

### Input

All inputs come from one Excel file (default path: `'/content/AR--1.xlsx'`). It must contain:

| Sheet name                        | Purpose         |
| --------------------------------- | --------------- |
| **Program Table Input 2 - Floor** | Floor metadata: |

* `Name`
* `Usable_Area` (SQM)
* `Max_Assignable_Floor_loading_Capacity`                                                 |
  \| **Program Table Input 1 - Block**    | Block metadata:
* `Block_ID`, `Block_Name`
* `Cumulative_Block_Circulation_Area` (SQM)
* `Max_Occupancy_with_Capacity`
* `Typical_Destination` (“Destination”/“Typical”/“both”)
* `Destination_Group`
* `SpaceMix_(ME_WE_US_Support_Speciality)`
* `Department_Sub-Department`                                                                |
  \| **Department Split**                 | Maps business units to sub-departments; column `BU_Department_Sub-Department` is renamed to
  `Department_Sub-Department`. First row skipped.                                              |
  \| **…Adjacency…**                      | Square matrix of adjacency scores (not used in core algorithm but parsed for future logic).        |
  \| **De-Centralized Logic**             | Rows labelled `Centralised`, `Semi Centralized`, `DeCentralised`, followed by an “( Add into …)”
  value specifying extra destination floors to include.                                            |

---

### Main Workflow

1. **Load & clean** each sheet, trim column names, coerce numeric types.

2. **Preprocess blocks**:

   * Split into two sets:

     * `destination_blocks` (`Typical_Destination` = “Destination” or “both”)
     * `typical_blocks`  (`Typical_Destination` = “Typical”)

3. **Initialize floor assignments** via

   ```python
   initialize_floor_assignments(floor_df: DataFrame) -> Dict[str, {
       remaining_area: float,
       remaining_capacity: int,
       assigned_blocks: List[dict],
       assigned_departments: Set[str],
       ME_area, WE_area, US_area, Support_area, Speciality_area: float
   }]
   ```

4. **Core stacking** in `run_stack_plan(mode: str)`:

   * **Mode**:

     * `"centralized"` → use 2 destination floors
     * `"semi"`        → 2 + extra from logic sheet
     * `"decentralized"`→ 2 + extra
   * **Phase 1:** Place **destination groups**

     1. Try to place each group wholesale on one of the first *max\_dest\_floors*.
     2. If that fails, try other floors.
     3. If still fails, split group block-by-block, greedily placing largest blocks first; leftover go to `unassigned_blocks`.
   * **Phase 2A:** Randomly assign all **ME** blocks (typical) into any floor with capacity.
   * **Phase 2B:** For each other category in `['WE','US','Support','Speciality']`:

     1. Compute target counts on each floor **proportional** to ME distribution.
     2. Shuffle blocks and place them to meet targets, falling back to any floor if needed; missing ones issued as warnings.
   * **Phase 3:** Build four Pandas DataFrames:

     1. `detailed_df`      – one row per placed block (floor, department, block name, group, space-mix, area, occupancy)
     2. `floor_summary_df` – per-floor totals (block count, sum area, sum occupancy) merged with original input values
     3. `space_mix_df`     – for each floor×category: unit counts and percentages of floor & overall
     4. `unassigned_df`    – blocks that couldn’t be placed

5. **Export** the three modes into Excel files, each with sheets `Detailed`, `Floor_Summary`, `SpaceMix_By_Units`, and `Unassigned`.

---

###  Functions Reference

#### `initialize_floor_assignments(floor_df)`

* **Input:** Floor DataFrame (must have columns `Name`, `Usable_Area`, `Max_Assignable_Floor_loading_Capacity`)
* **Output:**

  ```python
  {
    floor_name: {
      'remaining_area': float,
      'remaining_capacity': int,
      'assigned_blocks': [],
      'assigned_departments': set(),
      'ME_area': 0.0,  # likewise for WE_area, US_area, Support_area, Speciality_area
    },
    …
  }
  ```

#### `run_stack_plan(mode)`

```python
def run_stack_plan(mode: str) -> Tuple[
    pd.DataFrame,  # detailed_df
    pd.DataFrame,  # floor_summary_df
    pd.DataFrame,  # space_mix_df
    pd.DataFrame   # unassigned_df
]:
```

* **`mode`**: one of `'centralized'`, `'semi'`, `'decentralized'`
* **Returns:** the four dataframes described above.

---

### Outputs

* **Excel files** (in working directory):

  * `stack_plan_centralized16.xlsx`
  * `stack_plan_semi_centralized16.xlsx`
  * `stack_plan_decentralized16.xlsx`

Each contains four sheets:

* **Detailed**
* **Floor\_Summary**
* **SpaceMix\_By\_Units**
* **Unassigned**

---


# Logic



## 1. Destination‐Group Placement (Phase 1)

We treat each **Destination Group** G as a bundle of blocks with:

* Total area

  $$
    A_G = \sum_{b\in G} \text{area}(b)
  $$
* Total capacity

  $$
    C_G = \sum_{b\in G} \text{cap}(b)
  $$

and each floor f has remaining resources

$$
  R_f^{(A)},\;R_f^{(C)}.
$$

1. **Whole‐group fit**
   Try each floor f (first among the designated “destination floors,” then the rest) and check

   $$
     R_f^{(A)} \;\ge\; A_G
     \quad\text{and}\quad
     R_f^{(C)} \;\ge\; C_G.
   $$

   If true, assign *all* b∈G to floor f, and decrement

   $$
     R_f^{(A)} -= A_G,\quad R_f^{(C)} -= C_G.
   $$

2. **Split‐by‐trial**
   If no single floor can hold the entire group but the *sum* of all remaining areas across floors ≥ A\_G, we attempt a “trial placement”:

   * Sort G’s blocks in descending area.
   * Iteratively remove the largest block and test if the **reduced** group can fit as a whole (using the same check as above).
   * Once a reduced group **does** fit, we commit those assignments; the blocks we removed go back to being placed *one by one* (largest first) on whatever floor has enough room at that moment.
   * If even that fails, we skip straight to block‐by‐block placement for the entire group.

This greedy “largest‐first” strategy maximizes the chance of keeping groups intact, only peeling off the minimal number of biggest blocks.

---

## 2. Typical‐Block Assignment (Phase 2)

We categorize remaining blocks by their **SpaceMix** ∈ {ME, WE, US, Support, Speciality}.

### 2A. ME Blocks

* Shuffle all ME blocks randomly.
* For each block b, find the first floor f with

  $$
    R_f^{(A)} \;\ge\; \text{area}(b),
    \quad
    R_f^{(C)} \;\ge\; \text{cap}(b),
  $$

  assign b there, and decrement R\_f accordingly.
* Count the number of ME blocks on each floor:

  $$
    n_f^{(\mathrm{ME})} = \#\{b\!\in\!\text{ME} : b\text{ placed on }f\}.
  $$
* Total ME blocks: $N^{(\mathrm{ME})} = \sum_f n_f^{(\mathrm{ME})}$.
* Compute the *ME‐fraction* for each floor:

  $$
    \phi_f =
      \begin{cases}
        \tfrac{n_f^{(\mathrm{ME})}}{N^{(\mathrm{ME})}}, & N^{(\mathrm{ME})}>0,\\
        1/|\!F|, & \text{otherwise (uniform).}
      \end{cases}
  $$

### 2B. Other Categories

For each category X∈{WE,US,Support,Speciality}:

1. Let total blocks in X be $N^{(X)}$.
2. **Target counts** per floor:

   $$
     t_f^{(X)} = \text{round}\bigl(\phi_f\times N^{(X)}\bigr).
   $$
3. If $\sum_f t_f^{(X)}\neq N^{(X)}$, adjust by distributing the difference to floors with the largest fractional remainders of $\phi_f\times N^{(X)}$.
4. Shuffle X-blocks, then for each block b try to place it on any floor f that still has
   $\;R_f^{(A)}\ge\text{area}(b),\;R_f^{(C)}\ge\text{cap}(b)$ **and** still needs more X-blocks (i.e. current assigned < t\_f^{(X)}).
5. If a block can’t go to a floor with a deficit, fall back to *any* floor with capacity.
6. Otherwise, record it as unassigned.

---

## 3. Summary Metrics (Phase 3)

Once every assignment is done:

* **Detailed list:** one row per placed block with its floor, department, group, space-mix, area, and capacity.
* **Floor summary:** for each floor f,

  $$
    \begin{aligned}
      \text{Assgn\_Blocks}_f
        &= \#\{b\text{ on }f\},\\
      \text{Assgn\_Area}_f
        &= \sum_{b\text{ on }f}\text{area}(b),\\
      \text{Total\_Occupancy}_f
        &= \sum_{b\text{ on }f}\text{cap}(b).
    \end{aligned}
  $$
* **Space‐Mix percentages:** for each f and category X:

  $$
    \text{Pct\_Floor}_{f,X}
      = \frac{\#\{X\text{ on }f\}}{\sum_X \#\{X\text{ on }f\}}\times100\%,\quad
    \text{Pct\_Overall}_{f,X}
      = \frac{\#\{X\text{ on }f\}}{N^{(X)}}\times100\%.
  $$
* **Unassigned:** any blocks that couldn’t find a valid spot.

---


* **Capacity‐first grouping** keeps related “destination” spaces together as much as possible.
* **Proportional spread** (via the ME‐fraction φₙ) ensures that the other categories follow roughly the same floor-usage pattern, smoothing out peaks and valleys.
* **Greedy / largest‐first** fallbacks maximize utilization when perfect group‐placement isn’t possible.

we used **constraint checks**, **proportional targets**, and **greedy back‐off** to make algorithm robust across very different “modes” (centralized vs. distributed) and wildly varying block‐size profiles.


In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load Input Sheets
# ----------------------------------------

excel_path = '/content/AR--1.xlsx'  # ← adjust if needed

# 1.1 Floors sheet
all_floor_data = pd.read_excel(excel_path, sheet_name='Program Table Input 2 - Floor')
all_floor_data.columns = all_floor_data.columns.str.strip()

# 1.2 Blocks sheet
all_block_data = pd.read_excel(excel_path, sheet_name='Program Table Input 1 - Block')
all_block_data.columns = all_block_data.columns.str.strip()

# 1.3 Department Split sheet
department_split_data = pd.read_excel(excel_path, sheet_name='Department Split', skiprows=1)
department_split_data.columns = department_split_data.columns.str.strip()
department_split_data = department_split_data.rename(
    columns={'BU_Department_Sub-Department': 'Department_Sub-Department'}
)

# 1.4 Adjacency sheet
xls = pd.ExcelFile(excel_path)
adjacency_sheet_name = [name for name in xls.sheet_names if "Adjacency" in name][0]
raw_data = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_data.apply(pd.to_numeric, errors='coerce')
adjacency_data.index = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()

# 1.5 De-Centralized Logic sheet
# We expect rows: "Centralised", "Semi Centralized", "DeCentralised" in column 0,
# and their "( Add into cetralised destination Block)" values in column 1.
df_logic = pd.read_excel(excel_path, sheet_name='De-Centralized Logic', header=None)
De_Centralized_data = {}
current_section = None
for _, row in df_logic.iterrows():
    first_cell = str(row[0]).strip() if pd.notna(row[0]) else ""
    if first_cell in ["Centralised", "Semi Centralized", "DeCentralised"]:
        current_section = first_cell
        De_Centralized_data[current_section] = {"Add": 0}
    elif current_section and first_cell == "( Add into cetralised destination Block)":
        De_Centralized_data[current_section]["Add"] = int(row[1]) if pd.notna(row[1]) else 0

# Ensure keys exist
for key in ["Centralised", "Semi Centralized", "DeCentralised"]:
    if key not in De_Centralized_data:
        De_Centralized_data[key] = {"Add": 0}
    elif "Add" not in De_Centralized_data[key]:
        De_Centralized_data[key]["Add"] = 0

# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------

# 2.1 Convert cumulative circulation area from SQFT → SQM
#all_block_data['Cumulative_Area_SQM'] = (
 #   all_block_data['Cumulative_Block_Circulation_Area_(SQM)']
#)

# 2.2 Separate Destination vs. Typical blocks
destination_blocks = all_block_data[all_block_data['Typical_Destination'].isin(['Destination', 'both'])].copy()
typical_blocks = all_block_data[
    all_block_data['Typical_Destination'] == 'Typical'
].copy()

# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Returns a dict keyed by floor name. Each entry tracks:
      - remaining_area
      - remaining_capacity
      - assigned_blocks      (list of block‐row dicts)
      - assigned_departments (set of sub‐departments)
      - ME_area, WE_area, US_area, Support_area, Speciality_area (floats)
    """
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

floors = list(initialize_floor_assignments(all_floor_data).keys())

# ----------------------------------------
# Step 4: Core Stacking Function
# ----------------------------------------

def run_stack_plan(mode):
    """
    mode: 'centralized', 'semi', or 'decentralized'
    Returns four DataFrames:
      1) detailed_df      – each block’s assigned floor, department, block name, destination group, space mix, area, occupancy
      2) floor_summary_df – floor‐wise totals (block count, total area, total occupancy)
      3) space_mix_df     – for each floor and each category {ME, WE, US, Support, Speciality}:
                              - Unit_Count_on_Floor
                              - Pct_of_Floor_UC      = (category_count_on_floor / total_blocks_on_floor) × 100%
                              - Pct_of_Overall_UC    = (category_count_on_floor / total_blocks_of_category_overall) × 100%
      4) unassigned_df    – blocks that couldn’t be placed
    """
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = [] # Initialize unassigned_blocks here

    # Determine how many floors to use for destination blocks
    def destination_floor_count():
        if mode == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    max_dest_floors = destination_floor_count()
    # Cap at total number of floors
    max_dest_floors = min(max_dest_floors, len(floors))

    # Pre‐compute each group's total area and total capacity
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Block_Circulation_Area']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Phase 1: Assign destination groups (try whole‐group first; if that fails, split across floors)
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        info_grp = dest_groups[grp]
        grp_area = info_grp['total_area']
        grp_cap  = info_grp['total_capacity']
        placed_whole = False

        # 4.2.a Attempt to place entire group on any of the first max_dest_floors
        candidate_floors = floors[:max_dest_floors].copy()

        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                # Entire group fits here—place all blocks
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(
                        blk['Department_Sub_Department']
                    )
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # 4.2.b If not yet placed, try the remaining floors (beyond max_dest_floors)
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(
                            blk['Department_Sub_Department'].strip()
                        )
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # 4.2.c If still not placed as a whole, split the group block‐by‐block across floors
        if not placed_whole:
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # Try placing group by removing the largest blocks one-by-one until remaining can be placed whole
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Block_Circulation_Area'], reverse=True)
                removed_blocks = []
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    trial_area = sum(b['Cumulative_Block_Circulation_Area'] for b in trial_blocks)
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)

                    # Try to place this reduced group
                    floor_combination = []
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    temp_success = True
                    for blk in trial_blocks:
                        blk_area = blk['Cumulative_Block_Circulation_Area']
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity):
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        # Apply final assignment for successfully placed trial blocks
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Block_Circulation_Area']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        # Remove one largest block and retry
                        removed_blocks.append(trial_blocks.pop(0))

                # Place removed blocks one-by-one
                for blk in removed_blocks:
                    blk_area = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= bll_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                # Even splitting won't fit all blocks, place block-by-block
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    blk_area     = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False

                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)

    # Phase 2A: Randomly assign ME blocks (typical)
    me_blocks = [
        blk for blk in typical_blocks.to_dict('records')
        if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
    ]
    random.shuffle(me_blocks)
    for blk in me_blocks:
        blk_area     = blk['Cumulative_Block_Circulation_Area']
        blk_capacity = blk['Max_Occupancy_with_Capacity']
        blk_dept     = blk['Department_Sub_Department'].strip()

        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        placed = False
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            print(f"Warning: Could not place ME block '{blk['Block_Name']}'")

    # Compute ME distribution per floor (unit counts)
    me_count_per_floor = {fl: 0 for fl in floors}
    for fl, info in assignments.items():
        me_count_per_floor[fl] = sum(
            1 for blk in info['assigned_blocks']
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
        )
    total_me = sum(me_count_per_floor.values())
    if total_me == 0:
        me_frac_per_floor = {fl: 1 / len(floors) for fl in floors}
    else:
        me_frac_per_floor = {fl: me_count_per_floor[fl] / total_me for fl in floors}

    # Phase 2B: Assign other categories proportionally
    other_categories = ['WE', 'US', 'Support', 'Speciality']
    for category in other_categories:
        cat_blocks = [
            blk for blk in typical_blocks.to_dict('records')
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == category
        ]
        total_cat = len(cat_blocks)
        if total_cat == 0:
            continue

        raw_targets = {fl: me_frac_per_floor[fl] * total_cat for fl in floors}
        target_counts = {fl: int(round(raw_targets[fl])) for fl in floors}

        diff = total_cat - sum(target_counts.values())
        if diff != 0:
            fractional_parts = {
                fl: raw_targets[fl] - math.floor(raw_targets[fl]) for fl in floors
            }
            if diff > 0:
                for fl in sorted(floors, key=lambda x: fractional_parts[x], reverse=True)[:diff]:
                    target_counts[fl] += 1
            else:
                for fl in sorted(floors, key=lambda x: fractional_parts[x])[: -diff]:
                    target_counts[fl] -= 1

        random.shuffle(cat_blocks)
        assigned_counts = {fl: 0 for fl in floors}

        for blk in cat_blocks:
            blk_area     = blk['Cumulative_Block_Circulation_Area']
            blk_capacity = blk['Max_Occupancy_with_Capacity']
            blk_dept     = blk['Department_Sub_Department'].strip()

            deficits = {fl: target_counts[fl] - assigned_counts[fl] for fl in floors}
            floors_with_deficit = [fl for fl, d in deficits.items() if d > 0]
            if floors_with_deficit:
                candidate_floors = sorted(
                    floors_with_deficit,
                    key=lambda x: deficits[x],
                    reverse=True
                )
            else:
                candidate_floors = floors.copy()

            placed = False
            for fl in candidate_floors:
                if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area'] -= blk_area
                    assignments[fl]['remaining_capacity'] -= blk_capacity
                    assignments[fl]['assigned_departments'].add(blk_dept)
                    if category == 'WE':
                        assignments[fl]['WE_area'] += blk_area
                    elif category == 'US':
                        assignments[fl]['US_area'] += blk_area
                    elif category == 'Support':
                        assignments[fl]['Support_area'] += blk_area
                    elif category == 'Speciality':
                        assignments[fl]['Speciality_area'] += blk_area
                    assigned_counts[fl] += 1
                    placed = True
                    break

            if not placed:
                fallback = floors.copy()
                random.shuffle(fallback)
                for fl in fallback:
                    if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['remaining_area'] -= blk_area
                        assignments[fl]['remaining_capacity'] -= blk_capacity
                        assignments[fl]['assigned_departments'].add(blk_dept)
                        if category == 'WE':
                            assignments[fl]['WE_area'] += blk_area
                        elif category == 'US':
                            assignments[fl]['US_area'] += blk_area
                        elif category == 'Support':
                            assignments[fl]['Support_area'] += blk_area
                        elif category == 'Speciality':
                            assignments[fl]['Speciality_area'] += blk_area
                        assigned_counts[fl] += 1
                        placed = True
                        break

            if not placed:
                print(f"Warning: Could not place {category} block '{blk['Block_Name']}'")
                unassigned_blocks.append(blk)

    # Phase 3: Build Detailed & Summary DataFrames

    # 3.1 Detailed DataFrame
    assignment_list = []
    for fl, info in assignments.items():
        for blk in info['assigned_blocks']:
            assignment_list.append({
                'Block_id': blk['Block_ID'],
                'Floor': fl,
                'Department': blk['Department_Sub_Department'],
                'Block_Name': blk['Block_Name'],
                'Destination_Group': blk['Destination_Group'],
                'SpaceMix': blk['SpaceMix_(ME_WE_US_Support_Speciality)'],
                'Assigned_Area_SQM': blk['Cumulative_Block_Circulation_Area'],
                'Max_Occupancy': blk['Max_Occupancy_with_Capacity']
            })
    detailed_df = pd.DataFrame(assignment_list)

    # 3.2 Floor_Summary DataFrame
    # 4.6.1 Detailed DataFrame
    assignment_list = []
    for fl, info in assignments.items():
        for blk in info['assigned_blocks']:
            assignment_list.append({
                'Floor': fl,
                'Department': blk['Department_Sub_Department'],
                'Block_Name': blk['Block_Name'],
                'Destination_Group': blk['Destination_Group'],
                'SpaceMix': blk['SpaceMix_(ME_WE_US_Support_Speciality)'],
                'Assigned_Area_SQM': blk['Cumulative_Block_Circulation_Area'],
                'Max_Occupancy': blk['Max_Occupancy_with_Capacity']
            })
    detailed_df = pd.DataFrame(assignment_list)

    # 4.6.2 Floor_Summary DataFrame
     # 3.2 “Floor_Summary” DataFrame
    floor_summary_df = (
    detailed_df
    .groupby('Floor')
    .agg(
        Assgn_Blocks=('Block_Name', 'count'),
        Assgn_Area_SQM=('Assigned_Area_SQM', 'sum'),
        Total_Occupancy=('Max_Occupancy', 'sum')
    )
    .reset_index()
)

    # Merge with original floor input data to get base values
    floor_input_subset = all_floor_data[[
    'Name', 'Usable_Area', 'Max_Assignable_Floor_loading_Capacity'
]].rename(columns={
    'Name': 'Floor',
    'Usable_Area_(SQM)': 'Input_Usable_Area',
    'Max_Assignable_Floor_loading_Capacity': 'Input_Max_Capacity'
})

    # Join input data with summary
    floor_summary_df = pd.merge(
    floor_input_subset,
    floor_summary_df,
    on='Floor',
    how='left'
)

    # Fill NaNs (if any floor didn't get any assignments)
    floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]] = floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]].fillna(0)
    # 3.3 SpaceMix_By_Units DataFrame
    all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']
    category_totals = {
        cat: len(typical_blocks[
            typical_blocks['SpaceMix_(ME_WE_US_Support_Speciality)'].str.strip() == cat
        ])
        for cat in all_categories
    }

    rows = []
    for fl, info in assignments.items():
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            cat = blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
            if cat in counts:
                counts[cat] += 1
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            cnt = counts[cat]
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            total_cat = category_totals[cat]
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            rows.append({
                'Floor': fl,
                'SpaceMix': cat,
                'Unit_Count_on_Floor': cnt,
                'Pct_of_Floor_UC': round(pct_of_floor, 2),
                'Pct_of_Overall_UC': round(pct_overall, 2)
            })

    space_mix_df = pd.DataFrame(rows)

     # 4.6.4 Unassigned DataFrame
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),
            'Block_Name': blk.get('Block_Name', ''),
            'Destination_Group': blk.get('Destination_Group', ''),
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0)
        })
    unassigned_df = pd.DataFrame(unassigned_list)

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Generate & Export Three Excel Files
# ----------------------------------------
central_detailed, central_floor_sum, central_space_mix, central_unassigned = run_stack_plan('centralized')
semi_detailed, semi_floor_sum, semi_space_mix, semi_unassigned = run_stack_plan('semi')
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = run_stack_plan('decentralized')


with pd.ExcelWriter('stack_plan_centralized16.xlsx') as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


with pd.ExcelWriter('stack_plan_semi_centralized16.xlsx') as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


with pd.ExcelWriter('stack_plan_decentralized16.xlsx') as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


print("✅ Generated three Excel outputs:")
print("    • stack_plan_centralized8.xlsx")
print("    • stack_plan_semi_centralized8.xlsx")
print("    • stack_plan_decentralized8.xlsx")

✅ Generated three Excel outputs:
    • stack_plan_centralized8.xlsx
    • stack_plan_semi_centralized8.xlsx
    • stack_plan_decentralized8.xlsx


# AR-2

In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load All Input Sheets from AR-1.xlsx
# ----------------------------------------

excel_path = '/content/AR--2.xlsx'  # ← adjust if needed

# 1.1 Floors sheet (skip the first row)
all_floor_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 2 - Floor',
    skiprows=1
)
all_floor_data.columns = all_floor_data.columns.str.strip()
all_floor_data = all_floor_data.rename(columns={
    all_floor_data.columns[0]: 'Name',
    all_floor_data.columns[1]: 'Usable_Area_(SQM)',
    all_floor_data.columns[2]: 'Max_Assignable_Floor_loading_Capacity'
})

# Coerce floor‐area and capacity to numeric
#all_floor_data['Usable_Area_(SQM)'] = pd.to_numeric(
 #   all_floor_data['Usable_Area_(SQM)'], errors='raise'
#)
#all_floor_data['Max_Assignable_Floor_loading_Capacity'] = pd.to_numeric(
 #   all_foor_data['Max_Assignable_Floor_loading_Capacity'], errors='raise'
#)

# 1.2 Blocks sheet
all_block_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 1 - Block'
)
all_block_data.columns = all_block_data.columns.str.strip()

# Ensure these columns are numeric
#all_block_data['Cumulative_Area_SQM'] = pd.to_numeric(
 #   all_block_data['Cumulative_Block_Circulation_Area_(SQM)'], errors='raise'
#)
#all_block_data['Max_Occupancy_with_Capacity'] = pd.to_numeric(
#    all_block_data['Max_Occupancy_with_Capacity'], errors='raise'
#)

# 1.3 Department Split sheet (skip header row)
department_split_data = pd.read_excel(
    excel_path,
    sheet_name='Department Split',
    skiprows=1
)
department_split_data.columns = department_split_data.columns.str.strip()
department_split_data = department_split_data.rename(
    columns={'BU_Department_Sub-Department': 'Department_Sub-Department'}
)
# Build dictionaries:
#dept_splittable = department_split_data.set_index('Department_Sub-Department')['Splittable'].to_dict()
#dept_min_pct    = department_split_data.set_index('Department_Sub-Department')['Min_%_of_Block_per_department'].to_dict()

# 1.4 Min%Split sheet (not used below but loaded)
min_split_data = pd.read_excel(
    excel_path,
    sheet_name='Min % Split'
)
min_split_data.columns = min_split_data.columns.str.strip()

# 1.5 Adjacency sheet
xls = pd.ExcelFile(excel_path)
adjacency_sheet_name = [name for name in xls.sheet_names if "Adjacency" in name][0]
raw_data = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_data.apply(pd.to_numeric, errors='coerce')
adjacency_data.index   = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()

# 1.6 De-Centralized Logic sheet
df_logic = pd.read_excel(
    excel_path,
    sheet_name='De-Centralized Logic',
    header=None
)
De_Centralized_data = {}
current_section = None
for _, row in df_logic.iterrows():
    first_cell = str(row[0]).strip() if pd.notna(row[0]) else ""
    if first_cell in ["Centralised", "Semi Centralized", "DeCentralised"]:
        current_section = first_cell
        De_Centralized_data[current_section] = {"Add": 0}
    elif current_section and first_cell == "( Add into cetralised destination Block)":
        De_Centralized_data[current_section]["Add"] = int(row[1]) if pd.notna(row[1]) else 0

for key in ["Centralised", "Semi Centralized", "DeCentralised"]:
    if key not in De_Centralized_data:
        De_Centralized_data[key] = {"Add": 0}

# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------

# 2.2 Separate Destination vs. Typical blocks
destination_blocks = all_block_data[
    all_block_data['Typical_Destination'] == 'Destination'
].copy()
typical_blocks = all_block_data[
    all_block_data['Typical_Destination'] == 'Typical'
].copy()

# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Returns a dict keyed by floor name. Each entry has:
      - remaining_area
      - remaining_capacity
      - assigned_blocks      (list of block‐row dicts)
      - assigned_departments (set of sub‐departments)
      - ME_area, WE_area, US_area, Support_area, Speciality_area (floats)
    """
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area_(SQM)'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

floors = list(initialize_floor_assignments(all_floor_data).keys())

# ----------------------------------------
# Step 4: Core Stacking Function (with modified destination‐split logic + unassigned handling)
# ----------------------------------------

def run_stack_plan(mode):
    """
    mode: 'centralized', 'semi', or 'decentralized'
    Returns four DataFrames:
      detailed_df      – block‐to‐floor assignments
      floor_summary_df – floor totals (count, area, occupancy)
      space_mix_df     – for each floor & category {ME, WE, US, Support, Speciality}:
                          Unit_Count_on_Floor,
                          Pct_of_Floor_UC,
                          Pct_of_Overall_UC
      unassigned_df    – blocks that couldn’t be placed
    """
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []

    # 4.1 Determine how many floors to use for destination blocks
    def destination_floor_count():
        if mode == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    max_dest_floors = min(destination_floor_count(), len(floors))

    # 4.2 Group destination blocks by Destination_Group
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Area_SQM']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Phase 1: Assign destination groups (try whole‐group first; if that fails, split across floors)
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        info_grp = dest_groups[grp]
        grp_area = info_grp['total_area']
        grp_cap  = info_grp['total_capacity']
        placed_whole = False

        # 4.2.a Attempt to place entire group on any of the first max_dest_floors
        candidate_floors = floors[:max_dest_floors].copy()

        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                # Entire group fits here—place all blocks
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(
                        blk['Department_Sub_Department']
                    )
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # 4.2.b If not yet placed, try the remaining floors (beyond max_dest_floors)
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(
                            blk['Department_Sub_Department'].strip()
                        )
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # 4.2.c If still not placed as a whole, split the group block‐by‐block across floors
        if not placed_whole:
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # Try placing group by removing the largest blocks one-by-one until remaining can be placed whole
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True)
                removed_blocks = []
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    trial_area = sum(b['Cumulative_Area_SQM'] for b in trial_blocks)
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)

                    # Try to place this reduced group
                    floor_combination = []
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    temp_success = True
                    for blk in trial_blocks:
                        blk_area = blk['Cumulative_Area_SQM']
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity):
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        # Apply final assignment for successfully placed trial blocks
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Area_SQM']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        # Remove one largest block and retry
                        removed_blocks.append(trial_blocks.pop(0))

                # Place removed blocks one-by-one
                for blk in removed_blocks:
                    blk_area = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= bll_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                # Even splitting won't fit all blocks, place block-by-block
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    blk_area     = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False

                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)


    # Phase 2: Handle typical blocks with department‐splittable logic

    # 4.3 Separate typical blocks into:
    #   - dept_unsplittable_groups: {department → [block_dicts]} for Splittable != -1
    #   - splittable_blocks: list of block_dicts for Splittable == -1
    dept_unsplittable_groups = {}
    splittable_blocks = []

    for blk in typical_blocks.to_dict('records'):
        dept = blk['Department_Sub_Department'].strip()
        # ← DEFAULT TO -1 (splittable) IF MISSING
        spl = dept_splittable.get(dept, -1)
        if spl == -1:
            splittable_blocks.append(blk)
        else:
            dept_unsplittable_groups.setdefault(dept, []).append(blk)

    # 4.4 Phase 2A: Assign each unsplittable department's blocks as a group
    for dept, blocks_list in dept_unsplittable_groups.items():
        total_area = sum(b['Cumulative_Area_SQM'] for b in blocks_list)
        total_cap  = sum(b['Max_Occupancy_with_Capacity'] for b in blocks_list)
        placed = False

        candidate_floors = sorted(
            floors,
            key=lambda f: assignments[f]['remaining_area'],
            reverse=True
        )
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= total_area and
                assignments[fl]['remaining_capacity'] >= total_cap):
                for blk in blocks_list:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(dept)
                    cat = blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
                    if cat == 'ME':
                        assignments[fl]['ME_area'] += blk['Cumulative_Area_SQM']
                    elif cat == 'WE':
                        assignments[fl]['WE_area'] += blk['Cumulative_Area_SQM']
                    elif cat == 'US':
                        assignments[fl]['US_area'] += blk['Cumulative_Area_SQM']
                    elif cat.lower() == 'support':
                        assignments[fl]['Support_area'] += blk['Cumulative_Area_SQM']
                    elif cat.lower() == 'speciality':
                        assignments[fl]['Speciality_area'] += blk['Cumulative_Area_SQM']
                assignments[fl]['remaining_area'] -= total_area
                assignments[fl]['remaining_capacity'] -= total_cap
                placed = True
                break

        if not placed:
            # Mark entire department group as unassigned
            unassigned_blocks.extend(blocks_list)

    # 4.5 Phase 2B: On the remaining splittable blocks, assign by space‐mix logic

    # 4.5.a Assign all ME blocks randomly
    me_blocks = [
        blk for blk in splittable_blocks
        if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
    ]
    random.shuffle(me_blocks)
    for blk in me_blocks:
        blk_area     = blk['Cumulative_Area_SQM']
        blk_capacity = blk['Max_Occupancy_with_Capacity']
        blk_dept     = blk['Department_Sub_Department'].strip()

        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        placed = False
        for fl in candidate_floors:
            if assignments[fl]['remaining_area'] >= blk_area:
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            unassigned_blocks.append(blk)

    # 4.5.b Compute ME distribution per floor (unit counts)
    me_count_per_floor = {fl: 0 for fl in floors}
    for fl, info in assignments.items():
        me_count_per_floor[fl] = sum(
            1 for blk in info['assigned_blocks']
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
        )
    total_me = sum(me_count_per_floor.values())
    if total_me == 0:
        me_frac_per_floor = {fl: 1 / len(floors) for fl in floors}
    else:
        me_frac_per_floor = {
            fl: me_count_per_floor[fl] / total_me for fl in floors
        }

    # 4.5.c Assign other categories proportionally
    other_categories = ['WE', 'US', 'Support', 'Speciality']
    for category in other_categories:
        cat_blocks = [
            blk for blk in splittable_blocks
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == category
        ]
        total_cat = len(cat_blocks)
        if total_cat == 0:
            continue

        raw_targets = {fl: me_frac_per_floor[fl] * total_cat for fl in floors}
        target_counts = {fl: int(round(raw_targets[fl])) for fl in floors}

        diff = total_cat - sum(target_counts.values())
        if diff != 0:
            fractional_parts = {
                fl: raw_targets[fl] - math.floor(raw_targets[fl]) for fl in floors
            }
            if diff > 0:
                for fl in sorted(floors, key=lambda x: fractional_parts[x], reverse=True)[:diff]:
                    target_counts[fl] += 1
            else:
                for fl in sorted(floors, key=lambda x: fractional_parts[x])[: -diff]:
                    target_counts[fl] -= 1

        random.shuffle(cat_blocks)
        assigned_counts = {fl: 0 for fl in floors}

        for blk in cat_blocks:
            blk_area     = blk['Cumulative_Area_SQM']
            blk_capacity = blk['Max_Occupancy_with_Capacity']
            blk_dept     = blk['Department_Sub_Department'].strip()

            deficits = {fl: target_counts[fl] - assigned_counts[fl] for fl in floors}
            floors_with_deficit = [fl for fl, d in deficits.items() if d > 0]
            if floors_with_deficit:
                candidate_floors = sorted(
                    floors_with_deficit,
                    key=lambda x: deficits[x],
                    reverse=True
                )
            else:
                candidate_floors = floors.copy()

            placed = False
            for fl in candidate_floors:
                if assignments[fl]['remaining_area'] >= blk_area:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area'] -= blk_area
                    assignments[fl]['remaining_capacity'] -= blk_capacity
                    assignments[fl]['assigned_departments'].add(blk_dept)
                    if category == 'WE':
                        assignments[fl]['WE_area'] += blk_area
                    elif category == 'US':
                        assignments[fl]['US_area'] += blk_area
                    elif category == 'Support':
                        assignments[fl]['Support_area'] += blk_area
                    elif category == 'Speciality':
                        assignments[fl]['Speciality_area'] += blk_area
                    assigned_counts[fl] += 1
                    placed = True
                    break

            if not placed:
                # Try fallback random floors
                fallback = floors.copy()
                random.shuffle(fallback)
                for fl in fallback:
                    if assignments[fl]['remaining_area'] >= blk_area:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['remaining_area'] -= blk_area
                        assignments[fl]['remaining_capacity'] -= blk_capacity
                        assignments[fl]['assigned_departments'].add(blk_dept)
                        if category == 'WE':
                            assignments[fl]['WE_area'] += blk_area
                        elif category == 'US':
                            assignments[fl]['US_area'] += blk_area
                        elif category == 'Support':
                            assignments[fl]['Support_area'] += blk_area
                        elif category == 'Speciality':
                            assignments[fl]['Speciality_area'] += blk_area
                        assigned_counts[fl] += 1
                        placed = True
                        break

            if not placed:
                unassigned_blocks.append(blk)

    # 4.6 Phase 3: Build Output DataFrames

    # 4.6.1 Detailed DataFrame
    assignment_list = []
    for fl, info in assignments.items():
        for blk in info['assigned_blocks']:
            assignment_list.append({
                'Floor': fl,
                'Department': blk['Department_Sub_Department'],
                'Block_Name': blk['Block_Name'],
                'Destination_Group': blk['Destination_Group'],
                'SpaceMix': blk['SpaceMix_(ME_WE_US_Support_Speciality)'],
                'Assigned_Area_SQM': blk['Cumulative_Area_SQM'],
                'Max_Occupancy': blk['Max_Occupancy_with_Capacity']
            })
    detailed_df = pd.DataFrame(assignment_list)

    # 4.6.2 Floor_Summary DataFrame
     # 3.2 “Floor_Summary” DataFrame
    floor_summary_df = (
    detailed_df
    .groupby('Floor')
    .agg(
        Assgn_Blocks=('Block_Name', 'count'),
        Assgn_Area_SQM=('Assigned_Area_SQM', 'sum'),
        Total_Occupancy=('Max_Occupancy', 'sum')
    )
    .reset_index()
)

    # Merge with original floor input data to get base values
    floor_input_subset = all_floor_data[[
    'Name', 'Usable_Area_(SQM)', 'Max_Assignable_Floor_loading_Capacity'
]].rename(columns={
    'Name': 'Floor',
    'Usable_Area_(SQM)': 'Input_Usable_Area_SQM',
    'Max_Assignable_Floor_loading_Capacity': 'Input_Max_Capacity'
})

    # Join input data with summary
    floor_summary_df = pd.merge(
    floor_input_subset,
    floor_summary_df,
    on='Floor',
    how='left'
)

    # Fill NaNs (if any floor didn't get any assignments)
    floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]] = floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]].fillna(0)

    # 3.3 “SpaceMix_By_Units” DataFrame
    all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']
    category_totals = {
        cat: len(typical_blocks[
            typical_blocks['SpaceMix_(ME_WE_US_Support_Speciality)'].str.strip() == cat
        ])
        for cat in all_categories
    }

    rows = []
    for fl, info in assignments.items():
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            cat = blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
            if cat in counts:
                counts[cat] += 1
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            cnt = counts[cat]
            # Percent of floor’s blocks
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            # Percent of overall blocks of that category
            total_cat = category_totals[cat]
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            rows.append({
                'Floor': fl,
                'SpaceMix': cat,
                '%spaceMix': round(pct_overall, 2)

            })

    space_mix_df = pd.DataFrame(rows)

    # 4.6.4 Unassigned DataFrame
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),
            'Block_Name': blk.get('Block_Name', ''),
            'Destination_Group': blk.get('Destination_Group', ''),
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0)
        })
    unassigned_df = pd.DataFrame(unassigned_list)

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Generate & Export Excel + CSV Files (including Unassigned)
# ----------------------------------------

central_detailed, central_floor_sum, central_space_mix, central_unassigned = run_stack_plan('centralized')
semi_detailed,    semi_floor_sum,    semi_space_mix,    semi_unassigned    = run_stack_plan('semi')
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = run_stack_plan('decentralized')

# File names
central_file    = 'stack_plan_centralized28.xlsx'
semi_file       = 'stack_plan_semi_centralized28.xlsx'
decentral_file  = 'stack_plan_decentralized28.xlsx'

# --- ExcelWriter blocks with an extra sheet "Unassigned" ---
with pd.ExcelWriter(central_file) as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(semi_file) as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(decentral_file) as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

print("✅ Generated three Excel outputs (each with an 'Unassigned' sheet):")
print(f"    • {central_file}")
print(f"    • {semi_file}")
print(f"    • {decentral_file}")

# --- (Optional) Also export CSVs, if desired ---
#central_detailed.to_csv('stack_plan_centralized_detailed.csv', index=False)
#central_floor_sum.to_csv('stack_plan_centralized_floor_summary.csv', index=False)
#central_space_mix.to_csv('stack_plan_centralized_space_mix.csv', index=False)
#central_unassigned.to_csv('stack_plan_centralized_unassigned.csv', index=False)
#
#semi_detailed.to_csv('stack_plan_semi_centralized_detailed.csv', index=False)
#semi_floor_sum.to_csv('stack_plan_semi_centralized_floor_summary.csv', index=False)
#semi_space_mix.to_csv('stack_plan_semi_centralized_space_mix.csv', index=False)
#semi_unassigned.to_csv('stack_plan_semi_centralized_unassigned.csv', index=False)
#
#decentral_detailed.to_csv('stack_plan_decentralized_detailed.csv', index=False)
#decentral_floor_sum.to_csv('stack_plan_decentralized_floor_summary.csv', index=False)
#decentral_space_mix.to_csv('stack_plan_decentralized_space_mix.csv', index=False)
#decentral_unassigned.to_csv('stack_plan_decentralized_unassigned.csv', index=False)

KeyError: 'Usable_Area_(SQM)'

# BR_1,2


In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load All Input Sheets from AR-1.xlsx

# ----------------------------------------

excel_path = '/content/BR-1.xlsx'  # ← adjust if needed

# 1.1 Floors sheet (skip the first row)
all_floor_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 2 - Floor',
    skiprows=0  # Don't skip header
)
all_floor_data.columns = all_floor_data.columns.str.strip()
print(all_floor_data.columns.tolist())

all_floor_data = all_floor_data.rename(columns={
    all_floor_data.columns[0]: 'Name',
    all_floor_data.columns[1]: 'Usable_Area_(SQM)',
    all_floor_data.columns[2]: 'Max_Assignable_Floor_loading_Capacity'
})
print(all_floor_data.columns.tolist())
# Coerce floor‐area and capacity to numeric
all_floor_data['Usable_Area_(SQM)'] = pd.to_numeric(
    all_floor_data['Usable_Area_(SQM)'], errors='raise'
)
all_floor_data['Max_Assignable_Floor_loading_Capacity'] = pd.to_numeric(
    all_floor_data['Max_Assignable_Floor_loading_Capacity'], errors='raise'
)

# 1.2 Blocks sheet
all_block_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 1 - Block'
)
all_block_data.columns = all_block_data.columns.str.strip()

# Ensure these columns are numeric
all_block_data['Cumulative_Area_SQM'] = pd.to_numeric(
    all_block_data['Cumulative_Block_Circulation_Area'], errors='raise'
)
all_block_data['Max_Occupancy_with_Capacity'] = pd.to_numeric(
    all_block_data['Max_Occupancy_with_Capacity'], errors='raise'
)

# 1.3 Department Split sheet (skip header row)
department_split_data = pd.read_excel(
    excel_path,
    sheet_name='Department Split',
    skiprows=0 # Do not skip any rows initially
)
department_split_data.columns = department_split_data.iloc[0].str.strip() # Use the first row as header
department_split_data = department_split_data[1:].reset_index(drop=True) # Remove the first row (now header)

department_split_data.columns = department_split_data.columns.str.strip()
print(department_split_data.columns.tolist())
# Build dictionaries:
#dept_splittable = department_split_data.set_index('Department_Sub-Department')['Splittable'].to_dict()
#dept_min_pct    = department_split_data.set_index('Department_Sub-Department')['Min_%_of_Block_per_department'].to_dict()

# 1.4 Min%Split sheet (not used below but loaded)
min_split_data = pd.read_excel(
    excel_path,
    sheet_name='Min % Split'
)
min_split_data.columns = min_split_data.columns.str.strip()

# 1.5 Adjacency sheet
xls = pd.ExcelFile(excel_path)
adjacency_sheet_name = [name for name in xls.sheet_names if "Adjacency" in name][0]
raw_data = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_data.apply(pd.to_numeric, errors='coerce')
adjacency_data.index   = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()

# 1.6 De-Centralized Logic sheet
df_logic = pd.read_excel(
    excel_path,
    sheet_name='De-Centralized Logic',
    header=None
)
De_Centralized_data = {}
current_section = None
for _, row in df_logic.iterrows():
    first_cell = str(row[0]).strip() if pd.notna(row[0]) else ""
    if first_cell in ["Centralised", "Semi Centralized", "DeCentralised"]:
        current_section = first_cell
        De_Centralized_data[current_section] = {"Add": 0}
    elif current_section and first_cell == "( Add into cetralised destination Block)":
        De_Centralized_data[current_section]["Add"] = int(row[1]) if pd.notna(row[1]) else 0

for key in ["Centralised", "Semi Centralized", "DeCentralised"]:
    if key not in De_Centralized_data:
        De_Centralized_data[key] = {"Add": 0}

# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------

# 2.2 Separate Destination vs. Typical blocks
destination_blocks = all_block_data[
    all_block_data['Typical_Destination'] == 'Destination'
].copy()
typical_blocks = all_block_data[
    all_block_data['Typical_Destination'] == 'Typical'
].copy()

# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Returns a dict keyed by floor name. Each entry has:
      - remaining_area
      - remaining_capacity
      - assigned_blocks      (list of block‐row dicts)
      - assigned_departments (set of sub‐departments)
      - ME_area, WE_area, US_area, Support_area, Speciality_area (floats)
    """
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area_(SQM)'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

floors = list(initialize_floor_assignments(all_floor_data).keys())

# ----------------------------------------
# Step 4: Core Stacking Function (with modified destination‐split logic + unassigned handling)
# ----------------------------------------

def run_stack_plan(mode):
    """
    mode: 'centralized', 'semi', or 'decentralized'
    Returns four DataFrames:
      detailed_df      – block‐to‐floor assignments
      floor_summary_df – floor totals (count, area, occupancy)
      space_mix_df     – for each floor & category {ME, WE, US, Support, Speciality}:
                          Unit_Count_on_Floor,
                          Pct_of_Floor_UC,
                          Pct_of_Overall_UC
      unassigned_df    – blocks that couldn’t be placed
    """
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []

    # 4.1 Determine how many floors to use for destination blocks
    def destination_floor_count():
        if mode == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    max_dest_floors = min(destination_floor_count(), len(floors))

    # 4.2 Group destination blocks by Destination_Group
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Area_SQM']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Phase 1: Assign destination groups (try whole‐group first; if that fails, split across floors)
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        info_grp = dest_groups[grp]
        grp_area = info_grp['total_area']
        grp_cap  = info_grp['total_capacity']
        placed_whole = False

        # 4.2.a Attempt to place entire group on any of the first max_dest_floors
        candidate_floors = floors[:max_dest_floors].copy()

        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                # Entire group fits here—place all blocks
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(
                        blk['Department_Sub_Department']
                    )
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # 4.2.b If not yet placed, try the remaining floors (beyond max_dest_floors)
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(
                            blk['Department_Sub_Department'].strip()
                        )
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # 4.2.c If still not placed as a whole, split the group block‐by‐block across floors
        if not placed_whole:
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # Try placing group by removing the largest blocks one-by-one until remaining can be placed whole
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True)
                removed_blocks = []
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    trial_area = sum(b['Cumulative_Area_SQM'] for b in trial_blocks)
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)

                    # Try to place this reduced group
                    floor_combination = []
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    temp_success = True
                    for blk in trial_blocks:
                        blk_area = blk['Cumulative_Area_SQM']
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity):
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        # Apply final assignment for successfully placed trial blocks
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Area_SQM']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        # Remove one largest block and retry
                        removed_blocks.append(trial_blocks.pop(0))

                # Place removed blocks one-by-one
                for blk in removed_blocks:
                    blk_area = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                # Even splitting won't fit all blocks, place block-by-block
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    blk_area     = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False

                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)


    # Phase 2: Handle typical blocks with department‐splittable logic

    # 4.3 Separate typical blocks into:
    #   - dept_unsplittable_groups: {department → [block_dicts]} for Splittable != -1
    #   - splittable_blocks: list of block_dicts for Splittable == -1
    dept_unsplittable_groups = {}
    splittable_blocks = []

    for blk in typical_blocks.to_dict('records'):
        dept = blk['Department_Sub_Department'].strip()
        # ← DEFAULT TO -1 (splittable) IF MISSING
        spl = dept_splittable.get(dept, -1)
        if spl == -1:
            splittable_blocks.append(blk)
        else:
            dept_unsplittable_groups.setdefault(dept, []).append(blk)

    # 4.4 Phase 2A: Assign each unsplittable department's blocks as a group
    for dept, blocks_list in dept_unsplittable_groups.items():
        total_area = sum(b['Cumulative_Area_SQM'] for b in blocks_list)
        total_cap  = sum(b['Max_Occupancy_with_Capacity'] for b in blocks_list)
        placed = False

        candidate_floors = sorted(
            floors,
            key=lambda f: assignments[f]['remaining_area'],
            reverse=True
        )
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= total_area and
                assignments[fl]['remaining_capacity'] >= total_cap):
                for blk in blocks_list:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(dept)
                    cat = blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
                    if cat == 'ME':
                        assignments[fl]['ME_area'] += blk['Cumulative_Area_SQM']
                    elif cat == 'WE':
                        assignments[fl]['WE_area'] += blk['Cumulative_Area_SQM']
                    elif cat == 'US':
                        assignments[fl]['US_area'] += blk['Cumulative_Area_SQM']
                    elif cat.lower() == 'support':
                        assignments[fl]['Support_area'] += blk['Cumulative_Area_SQM']
                    elif cat.lower() == 'speciality':
                        assignments[fl]['Speciality_area'] += blk['Cumulative_Area_SQM']
                assignments[fl]['remaining_area'] -= total_area
                assignments[fl]['remaining_capacity'] -= total_cap
                placed = True
                break

        if not placed:
            # Mark entire department group as unassigned
            unassigned_blocks.extend(blocks_list)

    # 4.5 Phase 2B: On the remaining splittable blocks, assign by space‐mix logic

    # 4.5.a Assign all ME blocks randomly
    me_blocks = [
        blk for blk in splittable_blocks
        if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
    ]
    random.shuffle(me_blocks)
    for blk in me_blocks:
        blk_area     = blk['Cumulative_Area_SQM']
        blk_capacity = blk['Max_Occupancy_with_Capacity']
        blk_dept     = blk['Department_Sub_Department'].strip()

        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        placed = False
        for fl in candidate_floors:
            if assignments[fl]['remaining_area'] >= blk_area:
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            unassigned_blocks.append(blk)

    # 4.5.b Compute ME distribution per floor (unit counts)
    me_count_per_floor = {fl: 0 for fl in floors}
    for fl, info in assignments.items():
        me_count_per_floor[fl] = sum(
            1 for blk in info['assigned_blocks']
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
        )
    total_me = sum(me_count_per_floor.values())
    if total_me == 0:
        me_frac_per_floor = {fl: 1 / len(floors) for fl in floors}
    else:
        me_frac_per_floor = {
            fl: me_count_per_floor[fl] / total_me for fl in floors
        }

    # 4.5.c Assign other categories proportionally
    other_categories = ['WE', 'US', 'Support', 'Speciality']
    for category in other_categories:
        cat_blocks = [
            blk for blk in splittable_blocks
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == category
        ]
        total_cat = len(cat_blocks)
        if total_cat == 0:
            continue

        raw_targets = {fl: me_frac_per_floor[fl] * total_cat for fl in floors}
        target_counts = {fl: int(round(raw_targets[fl])) for fl in floors}

        diff = total_cat - sum(target_counts.values())
        if diff != 0:
            fractional_parts = {
                fl: raw_targets[fl] - math.floor(raw_targets[fl]) for fl in floors
            }
            if diff > 0:
                for fl in sorted(floors, key=lambda x: fractional_parts[x], reverse=True)[:diff]:
                    target_counts[fl] += 1
            else:
                for fl in sorted(floors, key=lambda x: fractional_parts[x])[: -diff]:
                    target_counts[fl] -= 1

        random.shuffle(cat_blocks)
        assigned_counts = {fl: 0 for fl in floors}

        for blk in cat_blocks:
            blk_area     = blk['Cumulative_Area_SQM']
            blk_capacity = blk['Max_Occupancy_with_Capacity']
            blk_dept     = blk['Department_Sub_Department'].strip()

            deficits = {fl: target_counts[fl] - assigned_counts[fl] for fl in floors}
            floors_with_deficit = [fl for fl, d in deficits.items() if d > 0]
            if floors_with_deficit:
                candidate_floors = sorted(
                    floors_with_deficit,
                    key=lambda x: deficits[x],
                    reverse=True
                )
            else:
                candidate_floors = floors.copy()

            placed = False
            for fl in candidate_floors:
                if assignments[fl]['remaining_area'] >= blk_area:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area'] -= blk_area
                    assignments[fl]['remaining_capacity'] -= blk_capacity
                    assignments[fl]['assigned_departments'].add(blk_dept)
                    if category == 'WE':
                        assignments[fl]['WE_area'] += blk_area
                    elif category == 'US':
                        assignments[fl]['US_area'] += blk_area
                    elif category == 'Support':
                        assignments[fl]['Support_area'] += blk_area
                    elif category == 'Speciality':
                        assignments[fl]['Speciality_area'] += blk_area
                    assigned_counts[fl] += 1
                    placed = True
                    break

            if not placed:
                # Try fallback random floors
                fallback = floors.copy()
                random.shuffle(fallback)
                for fl in fallback:
                    if assignments[fl]['remaining_area'] >= blk_area:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['remaining_area'] -= blk_area
                        assignments[fl]['remaining_capacity'] -= blk_capacity
                        assignments[fl]['assigned_departments'].add(blk_dept)
                        if category == 'WE':
                            assignments[fl]['WE_area'] += blk_area
                        elif category == 'US':
                            assignments[fl]['US_area'] += blk_area
                        elif category == 'Support':
                            assignments[fl]['Support_area'] += blk_area
                        elif category == 'Speciality':
                            assignments[fl]['Speciality_area'] += blk_area
                        assigned_counts[fl] += 1
                        placed = True
                        break

            if not placed:
                unassigned_blocks.append(blk)

    # 4.6 Phase 3: Build Output DataFrames

    # 4.6.1 Detailed DataFrame
    assignment_list = []
    for fl, info in assignments.items():
        for blk in info['assigned_blocks']:
            assignment_list.append({
                'Floor': fl,
                'Department': blk['Department_Sub_Department'],
                'Block_Name': blk['Block_Name'],
                'Destination_Group': blk['Destination_Group'],
                'SpaceMix': blk['SpaceMix_(ME_WE_US_Support_Speciality)'],
                'Assigned_Area_SQM': blk['Cumulative_Area_SQM'],
                'Max_Occupancy': blk['Max_Occupancy_with_Capacity']
            })
    detailed_df = pd.DataFrame(assignment_list)

    # 4.6.2 Floor_Summary DataFrame
     # 3.2 “Floor_Summary” DataFrame
    floor_summary_df = (
    detailed_df
    .groupby('Floor')
    .agg(
        Assgn_Blocks=('Block_Name', 'count'),
        Assgn_Area_SQM=('Assigned_Area_SQM', 'sum'),
        Total_Occupancy=('Max_Occupancy', 'sum')
    )
    .reset_index()
)

    # Merge with original floor input data to get base values
    floor_input_subset = all_floor_data[[
    'Name', 'Usable_Area_(SQM)', 'Max_Assignable_Floor_loading_Capacity'
]].rename(columns={
    'Name': 'Floor',
    'Usable_Area_(SQM)': 'Input_Usable_Area_SQM',
    'Max_Assignable_Floor_loading_Capacity': 'Input_Max_Capacity'
})

    # Join input data with summary
    floor_summary_df = pd.merge(
    floor_input_subset,
    floor_summary_df,
    on='Floor',
    how='left'
)

    # Fill NaNs (if any floor didn't get any assignments)
    floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]] = floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]].fillna(0)

    # 3.3 “SpaceMix_By_Units” DataFrame
    all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']
    category_totals = {
        cat: len(typical_blocks[
            typical_blocks['SpaceMix_(ME_WE_US_Support_Speciality)'].str.strip() == cat
        ])
        for cat in all_categories
    }

    rows = []
    for fl, info in assignments.items():
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            cat = blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
            if cat in counts:
                counts[cat] += 1
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            cnt = counts[cat]
            # Percent of floor’s blocks
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            # Percent of overall blocks of that category
            total_cat = category_totals[cat]
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            rows.append({
                'Floor': fl,
                'SpaceMix': cat,
                '%spaceMix': round(pct_overall, 2)

            })

    space_mix_df = pd.DataFrame(rows)

    # 4.6.4 Unassigned DataFrame
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),
            'Block_Name': blk.get('Block_Name', ''),
            'Destination_Group': blk.get('Destination_Group', ''),
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0)
        })
    unassigned_df = pd.DataFrame(unassigned_list)

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Generate & Export Excel + CSV Files (including Unassigned)
# ----------------------------------------

central_detailed, central_floor_sum, central_space_mix, central_unassigned = run_stack_plan('centralized')
semi_detailed,    semi_floor_sum,    semi_space_mix,    semi_unassigned    = run_stack_plan('semi')
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = run_stack_plan('decentralized')

# File names
central_file    = 'stack_plan_centralized28.xlsx'
semi_file       = 'stack_plan_semi_centralized28.xlsx'
decentral_file  = 'stack_plan_decentralized28.xlsx'

# --- ExcelWriter blocks with an extra sheet "Unassigned" ---
with pd.ExcelWriter(central_file) as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(semi_file) as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(decentral_file) as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

print("✅ Generated three Excel outputs (each with an 'Unassigned' sheet):")
print(f"    • {central_file}")
print(f"    • {semi_file}")
print(f"    • {decentral_file}")


# --- (Optional) Also export CSVs, if desired ---
#central_detailed.to_csv('stack_plan_centralized_detailed.csv', index=False)
#central_floor_sum.to_csv('stack_plan_centralized_floor_summary.csv', index=False)
#central_space_mix.to_csv('stack_plan_centralized_space_mix.csv', index=False)
#central_unassigned.to_csv('stack_plan_centralized_unassigned.csv', index=False)
#
#semi_detailed.to_csv('stack_plan_semi_centralized_detailed.csv', index=False)
#semi_floor_sum.to_csv('stack_plan_semi_centralized_floor_summary.csv', index=False)
#semi_space_mix.to_csv('stack_plan_semi_centralized_space_mix.csv', index=False)
#semi_unassigned.to_csv('stack_plan_semi_centralized_unassigned.csv', index=False)
#
#decentral_detailed.to_csv('stack_plan_decentralized_detailed.csv', index=False)
#decentral_floor_sum.to_csv('stack_plan_decentralized_floor_summary.csv', index=False)
#decentral_space_mix.to_csv('stack_plan_decentralized_space_mix.csv', index=False)
#decentral_unassigned.to_csv('stack_plan_decentralized_unassigned.csv', index=False)

['Name', 'Usable Area', 'Max Assignable Floor loading Capacity']
['Name', 'Usable_Area_(SQM)', 'Max_Assignable_Floor_loading_Capacity']
['PUBLIC SPACE >> FRONT OFFICE', 'CAO/SC BM', 'PUBLIC SPACE_FRONT OFFICE_CAO/SC BM', nan, nan]
✅ Generated three Excel outputs (each with an 'Unassigned' sheet):
    • stack_plan_centralized28.xlsx
    • stack_plan_semi_centralized28.xlsx
    • stack_plan_decentralized28.xlsx


In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load All Input Sheets from BR-2.xlsx
# ----------------------------------------
excel_path = '/content/BR-2.xlsx'  # adjust if needed

# 1.1 Floors sheet
all_floor_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 2 - Floor',
    skiprows=0
)
all_floor_data.columns = all_floor_data.columns.str.strip()
all_floor_data = all_floor_data.rename(columns={
    all_floor_data.columns[0]: 'Name',
    all_floor_data.columns[1]: 'Usable_Area_(SQM)',
    all_floor_data.columns[2]: 'Max_Assignable_Floor_loading_Capacity'
})
all_floor_data['Usable_Area_(SQM)'] = pd.to_numeric(
    all_floor_data['Usable_Area_(SQM)'], errors='raise'
)
all_floor_data['Max_Assignable_Floor_loading_Capacity'] = pd.to_numeric(
    all_floor_data['Max_Assignable_Floor_loading_Capacity'], errors='raise'
)

# 1.2 Blocks sheet
all_block_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 1 - Block'
)
all_block_data.columns = all_block_data.columns.str.strip()
all_block_data['Cumulative_Area_SQM'] = pd.to_numeric(
    all_block_data['Cumulative_Block_Circulation_Area'], errors='raise'
)
all_block_data['Max_Occupancy_with_Capacity'] = pd.to_numeric(
    all_block_data['Max_Occupancy_with_Capacity'], errors='raise'
)

# 1.3 Department Split sheet
department_split_data = pd.read_excel(
    excel_path,
    sheet_name='Department Split',
    skiprows=1
)
department_split_data.columns = department_split_data.columns.str.strip()
department_split_data = department_split_data.rename(
    columns={
        department_split_data.columns[2]: 'Department_Sub_Department',
        department_split_data.columns[3]: 'Splittable',
        department_split_data.columns[4]: 'Min_%_of_Block_per_department'
    }
)
dept_splittable = department_split_data.set_index('Department_Sub_Department')['Splittable'].to_dict()
dept_min_pct = department_split_data.set_index('Department_Sub_Department')['Min_%_of_Block_per_department'].to_dict()

# 1.4 Min % Split sheet
min_split_data = pd.read_excel(
    excel_path,
    sheet_name='Min % Split'
)
min_split_data.columns = min_split_data.columns.str.strip()

# 1.5 Adjacency sheet
xls = pd.ExcelFile(excel_path)
adjacency_sheet_name = [name for name in xls.sheet_names if 'Adjacency' in name][0]
raw_adj = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_adj.apply(pd.to_numeric, errors='coerce')
adjacency_data.index = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()
adj_lookup = adjacency_data.to_dict()

# 1.6 De-Centralized Logic sheet
df_logic = pd.read_excel(
    excel_path,
    sheet_name='De-Centralized Logic',
    header=None
)
De_Centralized_data = {}
current_section = None
for _, row in df_logic.iterrows():
    first_cell = str(row[0]).strip() if pd.notna(row[0]) else ''
    if first_cell in ['Centralised', 'Semi Centralized', 'DeCentralised']:
        current_section = first_cell
        De_Centralized_data[current_section] = {'Add': 0}
    elif current_section and 'Add' in str(first_cell):
        De_Centralized_data[current_section]['Add'] = int(row[1]) if pd.notna(row[1]) else 0
for key in ['Centralised', 'Semi Centralized', 'DeCentralised']:
    if key not in De_Centralized_data:
        De_Centralized_data[key] = {'Add': 0}

# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------
destination_blocks = all_block_data[all_block_data['Typical_Destination'] == 'Destination']
typical_blocks = all_block_data[all_block_data['Typical_Destination'] == 'Typical']

# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------
def initialize_floor_assignments(floor_df):
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area_(SQM)'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'DeptArea': {},  # track area per sub-dept
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

floors = list(initialize_floor_assignments(all_floor_data).keys())

# Helpers for adjacency & splitting
def can_place_block(blk, fl, assignments, mode):
    dept = blk['Department_Sub_Department'].strip()
    # 1) adjacency hard forbid
    for other in assignments[fl]['assigned_departments']:
        if adj_lookup[dept].get(other,0) == -1:
            return False
    # 2) destination floor lock (decentralized)
    if mode == 'decentralized' and blk['Typical_Destination']=='Destination':
        max_dest = 2 + De_Centralized_data['DeCentralised']['Add']
        if fl not in floors[:max_dest]:
            return False
    # 3) department split rules
    spl = dept_splittable.get(dept, -1)
    if spl == 1:
        # must stay on one floor
        for f2 in assignments:
            if f2!=fl and dept in assignments[f2]['assigned_departments']:
                return False
    elif spl == 0:
        # waterfall: enforce min pct first
        min_pct = dept_min_pct.get(dept,100)/100.0
        used = assignments[fl]['DeptArea'].get(dept,0.0) + blk['Cumulative_Area_SQM']
        floor_area = all_floor_data.loc[all_floor_data['Name']==fl,'Usable_Area_(SQM)'].iloc[0]
        if used/floor_area < min_pct:
            return False
    # -1 or 0.75: no hard block
    return True

def primary_category(blk):
    mix = blk['SpaceMix_(ME_WE_US_Support_Speciality)']
    for cat in ['ME','WE','US']:
        if cat in mix:
            return cat
    return 'Support'

# ----------------------------------------
# Step 4: Core Stacking Function
# ----------------------------------------
def run_stack_plan(mode):
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []
    max_dest_floors = min(2 + De_Centralized_data[
        'Semi Centralized' if mode=='semi' else (
            'DeCentralised' if mode=='decentralized' else 'Centralised'
        )]['Add'], len(floors))

    # Phase 1: Destination
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']
        dest_groups.setdefault(grp,{'blocks':[], 'total_area':0.0,'total_capacity':0}).update({})
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Area_SQM']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']
    names = list(dest_groups)
    random.shuffle(names)
    for grp in names:
        info = dest_groups[grp]
        placed = False
        # try floors[:max_dest_floors]
        for fl in floors[:max_dest_floors] + floors[max_dest_floors:]:
            if (assignments[fl]['remaining_area']>=info['total_area'] and
                assignments[fl]['remaining_capacity']>=info['total_capacity'] and
                all(can_place_block(b,fl,assignments,mode) for b in info['blocks'])):
                for blk in info['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                    assignments[fl]['DeptArea'][blk['Department_Sub_Department'].strip()] = \
                        assignments[fl]['DeptArea'].get(blk['Department_Sub_Department'].strip(),0)+blk['Cumulative_Area_SQM']
                assignments[fl]['remaining_area']    -= info['total_area']
                assignments[fl]['remaining_capacity']-= info['total_capacity']
                placed = True
                break
        if not placed:
            # fall back to individual
            for blk in sorted(info['blocks'],key=lambda b:b['Cumulative_Area_SQM'], reverse=True):
                for fl in sorted(floors, key=lambda x:assignments[x]['remaining_area'], reverse=True):
                    if (assignments[fl]['remaining_area']>=blk['Cumulative_Area_SQM'] and
                        assignments[fl]['remaining_capacity']>=blk['Max_Occupancy_with_Capacity'] and
                        can_place_block(blk,fl,assignments,mode)):
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                        assignments[fl]['DeptArea'][blk['Department_Sub_Department'].strip()] = \
                            assignments[fl]['DeptArea'].get(blk['Department_Sub_Department'].strip(),0)+blk['Cumulative_Area_SQM']
                        assignments[fl]['remaining_area']    -= blk['Cumulative_Area_SQM']
                        assignments[fl]['remaining_capacity']-= blk['Max_Occupancy_with_Capacity']
                        placed=True
                        break
                if not placed:
                    unassigned_blocks.append(blk)

    # Phase 2A: Unsplittable dept groups
    dept_groups = {}
    spl_blocks = []
    for blk in typical_blocks.to_dict('records'):
        dept = blk['Department_Sub_Department'].strip()
        if dept_splittable.get(dept,-1)==-1:
            spl_blocks.append(blk)
        else:
            dept_groups.setdefault(dept,[]).append(blk)
    for dept,blks in dept_groups.items():
        total_a = sum(b['Cumulative_Area_SQM'] for b in blks)
        total_c = sum(b['Max_Occupancy_with_Capacity'] for b in blks)
        placed=False
        for fl in sorted(floors,key=lambda f:assignments[f]['remaining_area'],reverse=True):
            if (assignments[fl]['remaining_area']>=total_a and
                assignments[fl]['remaining_capacity']>=total_c and
                all(can_place_block(b,fl,assignments,mode) for b in blks)):
                for blk in blks:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(dept)
                    cat=primary_category(blk)
                    assignments[fl][cat+'_area']+=blk['Cumulative_Area_SQM']
                    assignments[fl]['DeptArea'][dept]=assignments[fl]['DeptArea'].get(dept,0)+blk['Cumulative_Area_SQM']
                assignments[fl]['remaining_area']-=total_a
                assignments[fl]['remaining_capacity']-=total_c
                placed=True
                break
        if not placed:
            unassigned_blocks.extend(blks)

    # Phase 2B: Splittable blocks by space-mix
    # ME first
    me_blks = [b for b in spl_blocks if primary_category(b)=='ME']
    random.shuffle(me_blks)
    for blk in me_blks:
        for fl in random.sample(floors,len(floors)):
            if (assignments[fl]['remaining_area']>=blk['Cumulative_Area_SQM'] and
                assignments[fl]['remaining_capacity']>=blk['Max_Occupancy_with_Capacity'] and
                can_place_block(blk,fl,assignments,mode)):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area']    -= blk['Cumulative_Area_SQM']
                assignments[fl]['remaining_capacity']-= blk['Max_Occupancy_with_Capacity']
                assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                assignments[fl]['ME_area'] += blk['Cumulative_Area_SQM']
                assignments[fl]['DeptArea'][blk['Department_Sub_Department'].strip()]=assignments[fl]['DeptArea'].get(blk['Department_Sub_Department'].strip(),0)+blk['Cumulative_Area_SQM']
                break
        else:
            unassigned_blocks.append(blk)

    # compute target distribution
    me_counts = {f: sum(1 for b in assignments[f]['assigned_blocks'] if primary_category(b)=='ME') for f in floors}
    tot_me = sum(me_counts.values()) or 1
    frac_me = {f:me_counts[f]/tot_me for f in floors}

    for cat in ['WE','US','Support','Speciality']:
        cat_blks = [b for b in spl_blocks if primary_category(b)==cat]
        total_cat = len(cat_blks)
        if total_cat==0: continue
        raw_t = {f:frac_me[f]*total_cat for f in floors}
        tgt = {f:int(round(raw_t[f])) for f in floors}
        diff = total_cat - sum(tgt.values())
        if diff>0:
            frac_parts = {f:raw_t[f]-math.floor(raw_t[f]) for f in floors}
            for f in sorted(floors, key=lambda x:frac_parts[x], reverse=True)[:diff]: tgt[f]+=1
        elif diff<0:
            frac_parts = {f:raw_t[f]-math.floor(raw_t[f]) for f in floors}
            for f in sorted(floors, key=lambda x:frac_parts[x])[: -diff]: tgt[f]-=1
        random.shuffle(cat_blks)
        assigned={f:0 for f in floors}
        for blk in cat_blks:
            placed=False
            deficits={f:tgt[f]-assigned[f] for f in floors}
            cand = [f for f in floors if deficits[f]>0] or floors
            for fl in sorted(cand, key=lambda x:deficits.get(x,0), reverse=True):
                if (assignments[fl]['remaining_area']>=blk['Cumulative_Area_SQM'] and
                    assignments[fl]['remaining_capacity']>=blk['Max_Occupancy_with_Capacity'] and
                    can_place_block(blk,fl,assignments,mode)):
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area']    -= blk['Cumulative_Area_SQM']
                    assignments[fl]['remaining_capacity']-= blk['Max_Occupancy_with_Capacity']
                    assignments[fl][cat+'_area']+=blk['Cumulative_Area_SQM']
                    assignments[fl]['DeptArea'][blk['Department_Sub_Department'].strip()]=assignments[fl]['DeptArea'].get(blk['Department_Sub_Department'].strip(),0)+blk['Cumulative_Area_SQM']
                    assigned[fl]+=1
                    placed=True
                    break
            if not placed:
                unassigned_blocks.append(blk)

    # Build output DataFrames
    rows=[]
    for fl,info in assignments.items():
        for blk in info['assigned_blocks']:
            rows.append({
                'Floor':fl,
                'Department':blk['Department_Sub_Department'],
                'Block_Name':blk['Block_Name'],
                'Destination_Group':blk['Destination_Group'],
                'SpaceMix':primary_category(blk),
                'Assigned_Area_SQM':blk['Cumulative_Area_SQM'],
                'Max_Occupancy':blk['Max_Occupancy_with_Capacity']
            })
    detailed_df = pd.DataFrame(rows)
    summary = (
        detailed_df.groupby('Floor')
        .agg(Assgn_Blocks=('Block_Name','count'),
             Assgn_Area_SQM=('Assigned_Area_SQM','sum'),
             Total_Occupancy=('Max_Occupancy','sum'))
        .reset_index()
    )
    input_sub = all_floor_data[['Name','Usable_Area_(SQM)','Max_Assignable_Floor_loading_Capacity']].rename(columns={'Name':'Floor','Usable_Area_(SQM)':'Input_Usable_Area_SQM','Max_Assignable_Floor_loading_Capacity':'Input_Max_Capacity'})
    floor_summary_df = pd.merge(input_sub, summary, on='Floor', how='left').fillna(0)

    # space-mix by %
    cats=['ME','WE','US','Support','Speciality']
    cat_totals={cat:len([b for b in typical_blocks.to_dict('records') if primary_category(b)==cat]) for cat in cats}
    mix_rows=[]
    for fl,info in assignments.items():
        cnts={cat:0 for cat in cats}
        for blk in info['assigned_blocks']:
            cnts[primary_category(blk)]+=1
        tot_blocks=sum(cnts.values()) or 1
        for cat in cats:
            pct_overall = cnts[cat]/cat_totals[cat]*100 if cat_totals[cat] else 0
            mix_rows.append({'Floor':fl,'SpaceMix':cat,'%spaceMix':round(pct_overall,2)})
    space_mix_df = pd.DataFrame(mix_rows)

    un_rows=[{
        'Department':blk.get('Department_Sub_Department',''),
        'Block_Name':blk.get('Block_Name',''),
        'Destination_Group':blk.get('Destination_Group',''),
        'SpaceMix':primary_category(blk),
        'Area_SQM':blk.get('Cumulative_Area_SQM',0),
        'Max_Occupancy':blk.get('Max_Occupancy_with_Capacity',0)
    } for blk in unassigned_blocks]
    unassigned_df = pd.DataFrame(un_rows)

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Export
# ----------------------------------------
for mode,label in [('centralized','centralized'),('semi','semi_centralized'),('decentralized','decentralized')]:
    det, summ, mix, un = run_stack_plan(mode)
    fname = f'stack_plan_{label}28.xlsx'
    with pd.ExcelWriter(fname) as w:
        det.to_excel(w,'Detailed',index=False)
        summ.to_excel(w,'Floor_Summary',index=False)
        mix.to_excel(w,'SpaceMix_By_Units',index=False)
        un.to_excel(w,'Unassigned',index=False)
    print(f'Generated {fname}')

KeyError: 'Common_Common_Common'

In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load All Input Sheets from BR-2.xlsx
# ----------------------------------------
excel_path = '/content/AA-R1.xlsx'  # adjust if needed

# Load all sheets once
xls = pd.ExcelFile(excel_path)

# 1.1 Renovation Blocks sheet
renovation_block_data = xls.parse('RenovationProgram Table Input 1')
renovation_block_data.columns = renovation_block_data.columns.str.strip()

# Rename columns to match expected names used later in the script
renovation_block_data = renovation_block_data.rename(columns={
    'Cumulative_Block_Circulation_Area': 'Cumulative_Area_SQM',
    'Max_Occupancy_with_Capacity': 'Max_Occupancy_with_Capacity' # This column name seems correct already
})




# Ensure relevant columns are numeric (using the potentially renamed columns)
for df in [renovation_block_data, existing_block_data]:
    df['Cumulative_Area_SQM'] = pd.to_numeric(
        df['Cumulative_Area_SQM'], errors='coerce' # Use coerce to handle potential non-numeric values
    ).fillna(0) # Fill NaN with 0 after coercion
    df['Max_Occupancy_with_Capacity'] = pd.to_numeric(
        df['Max_Occupancy_with_Capacity'], errors='coerce' # Use coerce
    ).fillna(0) # Fill NaN with 0

# Combine renovation and existing blocks for a complete set of blocks to be placed or considered
all_block_data = pd.concat([renovation_block_data], ignore_index=True)


# 1.3 Floors sheet
all_floor_data = xls.parse('Program Table Input 2 - Floor', skiprows=0) # Don't skip header
all_floor_data.columns = all_floor_data.columns.str.strip()
print(all_floor_data.columns.tolist())

all_floor_data = all_floor_data.rename(columns={
    all_floor_data.columns[0]: 'Name',
    all_floor_data.columns[1]: 'Usable_Area_(SQM)',
    all_floor_data.columns[2]: 'Max_Assignable_Floor_loading_Capacity'
})
print(all_floor_data.columns.tolist())
# Coerce floor‐area and capacity to numeric
all_floor_data['Usable_Area_(SQM)'] = pd.to_numeric(
    all_floor_data['Usable_Area_(SQM)'], errors='coerce' # Use coerce
).fillna(0) # Fill NaN with 0
all_floor_data['Max_Assignable_Floor_loading_Capacity'] = pd.to_numeric(
    all_floor_data['Max_Assignable_Floor_loading_Capacity'], errors='coerce' # Use coerce
).fillna(0) # Fill NaN with 0


# 1.4 Department Split sheet (skip header row)
department_split_data = xls.parse('Department Split', header=1) # Set the second row as the header
department_split_data.columns = department_split_data.columns.str.strip()
print(department_split_data.columns.tolist())

# Correctly rename the columns based on the current output
department_split_data = department_split_data.rename(
    columns={
        department_split_data.columns[0]: 'Department_Sub-Department', # The third column is the correct one
        department_split_data.columns[1]: 'Splittable', # Assuming the fourth column is Splittable
        department_split_data.columns[2]: 'Min_%of_Block_per_department' # Assuming the fifth column is Min%_of_Block_per_department
    }
)

print(department_split_data.columns.tolist())
# Build dictionaries:
dept_splittable = department_split_data.set_index('Department_Sub-Department')['Splittable'].to_dict()
dept_min_pct    = department_split_data.set_index('Department_Sub-Department')['Min_%of_Block_per_department'].to_dict()

# 1.5 Min%Split sheet (not used below but loaded)
min_split_data = xls.parse('Min % Split')
min_split_data.columns = min_split_data.columns.str.strip()

# 1.6 Adjacency sheet
adjacency_sheet_name = [name for name in xls.sheet_names if "Adjacency" in name][0]
raw_adj = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_adj.apply(pd.to_numeric, errors='coerce')
adjacency_data.index = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()
adj_lookup = adjacency_data.to_dict()

# 1.7 De-Centralized Logic sheet
df_logic = xls.parse('De-Centralized Logic', header=None)
De_Centralized_data = {}
current_section = None
for _, row in df_logic.iterrows():
    first_cell = str(row[0]).strip() if pd.notna(row[0]) else ''
    if first_cell in ['Centralised', 'Semi Centralized', 'DeCentralised']:
        current_section = first_cell
        De_Centralized_data[current_section] = {'Add': 0}
    elif current_section and 'Add' in str(first_cell):
        De_Centralized_data[current_section]['Add'] = int(row[1]) if pd.notna(row[1]) else 0
for key in ['Centralised', 'Semi Centralized', 'DeCentralised']:
    if key not in De_Centralized_data:
        De_Centralized_data[key] = {'Add': 0}

# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------

# 2.1 Separate Immovable vs. Movable blocks
immovable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'] == 'Immovable Asset'
].copy()
movable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'] == 'Movable Asset'
].copy()

# 2.2 Separate Destination vs. Typical blocks from movable blocks
destination_blocks = movable_blocks[
    movable_blocks['Typical_Destination'] == 'Destination'
].copy()
typical_blocks = movable_blocks[
    movable_blocks['Typical_Destination'] == 'Typical'
].copy()

# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Returns a dict keyed by floor name. Each entry has:
      - remaining_area
      - remaining_capacity
      - assigned_blocks      (list of block‐row dicts)
      - assigned_departments (set of sub‐departments)
      - ME_area, WE_area, US_area, Support_area, Speciality_area (floats)
    """
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area_(SQM)'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'DeptArea': {},  # track area per sub-dept
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

floors = list(initialize_floor_assignments(all_floor_data).keys())

# Define all_categories outside the function
all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']

# Helpers for adjacency & splitting
def can_place_block(blk, fl, assignments, mode):
    dept = blk.get('Department_Sub_Department', '').strip() # Use .get for robustness
    # 1) adjacency hard forbid
    for other in assignments[fl]['assigned_departments']:
        if dept in adj_lookup and other in adj_lookup[dept] and adj_lookup[dept][other] == -1:
            return False
    # 2) destination floor lock (decentralized)
    if mode == 'decentralized' and blk.get('Typical_Destination','')=='Destination': # Use .get for robustness
        max_dest = 2 + De_Centralized_data['DeCentralised']['Add']
        if fl not in floors[:max_dest]:
            return False
    # 3) department split rules
    spl = dept_splittable.get(dept, -1)
    if spl == 1:
        # must stay on one floor
        for f2 in assignments:
            if f2!=fl and dept in assignments[f2]['assigned_departments']:
                return False
    elif spl == 0:
        # waterfall: enforce min pct first
        min_pct = dept_min_pct.get(dept,100)/100.0
        used = assignments[fl]['DeptArea'].get(dept,0.0) + blk.get('Cumulative_Area_SQM', 0) # Use .get for robustness
        floor_area = all_floor_data.loc[all_floor_data['Name']==fl,'Usable_Area_(SQM)'].iloc[0]
        if used/floor_area < min_pct:
            return False
    # -1 or 0.75: no hard block
    return True

def primary_category(blk):
    mix = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '') # Use .get for safety
    for cat in ['ME','WE','US']:
        if cat in mix:
            return cat
    return 'Support' # Default to Support if no match

# ----------------------------------------
# Step 4: Core Stacking Function
# ----------------------------------------
def run_stack_plan(mode):
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []

    # Phase 0: Pre-assign immovable blocks with constraint checks
    for _, blk_series in immovable_blocks.iterrows():
        blk = blk_series.to_dict() # Convert row to dictionary for consistent access
        assigned_level = blk.get('Level', '').strip()
        blk_area = blk.get('Cumulative_Area_SQM', 0)
        blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0)

        # Find the correct floor name based on the Level column
        matching_floor = None
        for floor_name in floors:
            if assigned_level.lower() in floor_name.lower():
                matching_floor = floor_name
                break

        if matching_floor and assignments[matching_floor]['remaining_area'] >= blk_area and assignments[matching_floor]['remaining_capacity'] >= blk_capacity:
            assignments[matching_floor]['assigned_blocks'].append(blk)
            assignments[matching_floor]['remaining_area'] -= blk_area
            assignments[matching_floor]['remaining_capacity'] -= blk_capacity
            assignments[matching_floor]['assigned_departments'].add(blk.get('Department_Sub_Department', '').strip())
            cat = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip()
            if cat == 'ME':
                assignments[matching_floor]['ME_area'] += blk_area
            elif cat == 'WE':
                assignments[matching_floor]['WE_area'] += blk_area
            elif cat == 'US':
                assignments[matching_floor]['US_area'] += blk_area
            elif cat.lower() == 'support':
                assignments[matching_floor]['Support_area'] += blk_area
            elif cat.lower() == 'speciality':
                assignments[matching_floor]['Speciality_area'] += blk_area
        else:
            # If an immovable block cannot be placed on its specified floor due to constraints
            unassigned_blocks.append(blk)


    # 4.1 Determine how many floors to use for destination blocks
    def destination_floor_count():
        if mode == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    max_dest_floors = min(destination_floor_count(), len(floors))

    # 4.2 Group destination blocks by Destination_Group
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Area_SQM']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Phase 1: Assign destination groups (try whole‐group first; if that fails, split across floors)
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        info_grp = dest_groups[grp]
        grp_area = info_grp['total_area']
        grp_cap  = info_grp['total_capacity']
        placed_whole = False

        # 4.2.a Attempt to place entire group on any of the first max_dest_floors
        candidate_floors = floors[:max_dest_floors].copy()

        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                # Entire group fits here—place all blocks
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(
                        blk['Department_Sub_Department']
                    )
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # 4.2.b If not yet placed, try the remaining floors (beyond max_dest_floors)
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(
                            blk['Department_Sub_Department'].strip()
                        )
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # 4.2.c If still not placed as a whole, split the group block‐by‐block across floors
        if not placed_whole:
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # Try placing group by removing the largest blocks one-by-one until remaining can be placed whole
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True)
                removed_blocks = []
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    trial_area = sum(b['Cumulative_Area_SQM'] for b in trial_blocks)
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)

                    # Try to place this reduced group
                    floor_combination = []
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    temp_success = True
                    for blk in trial_blocks:
                        blk_area = blk['Cumulative_Area_SQM']
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity and
                                can_place_block(blk,fl,temp_assignments,mode)): # Added can_place_block check
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        # Apply final assignment for successfully placed trial blocks
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Area_SQM']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        # Remove one largest block and retry
                        removed_blocks.append(trial_blocks.pop(0))

                # Place removed blocks one-by-one
                for blk in removed_blocks:
                    blk_area = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity and
                            can_place_block(blk,fl,assignments,mode)): # Added can_place_block check
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                # Even splitting won't fit all blocks, place block-by-block
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    blk_area     = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False

                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity and
                            can_place_block(blk,fl,assignments,mode)): # Added can_place_block check
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)


    # Phase 2: Handle typical blocks with department‐splittable logic

    # 4.3 Separate typical blocks into:
    #   - dept_unsplittable_groups: {department → [block_dicts]} for Splittable != -1
    #   - splittable_blocks: list of block_dicts for Splittable == -1
    dept_unsplittable_groups = {}
    splittable_blocks = []

    for blk in typical_blocks.to_dict('records'):
        dept = blk.get('Department_Sub_Department', '').strip() # Use .get for safety
        # ← DEFAULT TO -1 (splittable) IF MISSING
        spl = dept_splittable.get(dept, -1)
        if spl == 1:
            # must stay on one floor
            for f2 in assignments:
                if f2!=fl and dept in assignments[f2]['assigned_departments']:
                    return False
        else:
            dept_unsplittable_groups.setdefault(dept, []).append(blk)

    # 4.4 Phase 2A: Assign each unsplittable department's blocks as a group
    for dept, blocks_list in dept_unsplittable_groups.items():
        total_a = sum(b.get('Cumulative_Area_SQM', 0) for b in blocks_list) # Use .get for safety
        total_c = sum(b.get('Max_Occupancy_with_Capacity', 0) for b in blocks_list) # Use .get for safety
        placed=False

        candidate_floors = sorted(
            floors,
            key=lambda f: assignments[f]['remaining_area'],
            reverse=True
        )
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area']>=total_a and
                assignments[fl]['remaining_capacity']>=total_c and
                all(can_place_block(b,fl,assignments,mode) for b in blocks_list)): # Added can_place_block check
                for blk in blocks_list:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(dept)
                    cat=primary_category(blk)
                    assignments[fl][cat+'_area']+=blk.get('Cumulative_Area_SQM', 0) # Use .get for safety
                    assignments[fl]['DeptArea'][dept]=assignments[fl]['DeptArea'].get(dept,0)+blk.get('Cumulative_Area_SQM', 0) # Use .get for safety
                assignments[fl]['remaining_area']-=total_a
                assignments[fl]['remaining_capacity']-=total_c
                placed=True
                break

        if not placed:
            # Mark entire department group as unassigned
            unassigned_blocks.extend(blocks_list)

    # 4.5 Phase 2B: On the remaining splittable blocks, assign by space‐mix logic

    # 4.5.a Assign all ME blocks randomly
    me_blks = [
        blk for blk in splittable_blocks
        if blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip() == 'ME'
    ]
    random.shuffle(me_blks)
    for blk in me_blks:
        blk_area     = blk.get('Cumulative_Area_SQM', 0) # Use .get for safety
        blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0) # Use .get for safety
        blk_dept     = blk.get('Department_Sub-Department', '').strip() # Use .get for safety


        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        placed = False
        for fl in candidate_floors:
            if assignments[fl]['remaining_area'] >= blk_area:
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= bll_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            unassigned_blocks.append(blk)

    # compute target distribution
    me_counts = {fl: sum(1 for blk in assignments[fl]['assigned_blocks'] if primary_category(blk)=='ME') for fl in floors} # Use primary_category helper
    tot_me = sum(me_counts.values()) or 1
    frac_me = {f:me_counts[f]/tot_me for f in floors}

    for cat in ['WE','US','Support','Speciality']:
        cat_blks = [b for b in splittable_blocks if primary_category(b)==cat]
        total_cat = len(cat_blks)
        if total_cat==0: continue
        raw_t = {f:frac_me[f]*total_cat for f in floors}
        tgt = {f:int(round(raw_t[f])) for f in floors}
        diff = total_cat - sum(tgt.values())
        if diff>0:
            frac_parts = {f:raw_t[f]-math.floor(raw_t[f]) for f in floors}
            for f in sorted(floors, key=lambda x:frac_parts[x], reverse=True)[:diff]: tgt[f]+=1
        elif diff<0:
            frac_parts = {f:raw_t[f]-math.floor(raw_t[f]) for f in floors}
            for f in sorted(floors, key=lambda x:frac_parts[x])[: -diff]: tgt[f]-=1
        random.shuffle(cat_blks)
        assigned={f:0 for f in floors}
        for blk in cat_blks:
            placed=False
            blk_area = blk.get('Cumulative_Area_SQM', 0) # Use .get for safety
            blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0) # Use .get for safety
            blk_dept = blk.get('Department_Sub-Department', '').strip() # Use .get for safety

            deficits={f:tgt[f]-assigned[f] for f in floors}
            cand = [f for f in floors if deficits[f]>0] or floors
            for fl in sorted(cand, key=lambda x:deficits.get(x,0), reverse=True):
                if (assignments[fl]['remaining_area']>=blk_area and
                    assignments[fl]['remaining_capacity']>=blk_capacity and
                    can_place_block(blk,fl,assignments,mode)): # Added can_place_block check
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area']    -= blk_area
                    assignments[fl]['remaining_capacity']-= blk_capacity
                    assignments[fl][cat+'_area']+=blk_area
                    assigned[fl]+=1
                    placed=True
                    break
            if not placed:
                unassigned_blocks.append(blk)

    # Phase 3: Attempt to assign remaining unassigned blocks
    still_unassigned = []
    for blk in unassigned_blocks:
        blk_area = blk.get('Cumulative_Area_SQM', 0)
        blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0)
        placed = False
        # Try to place on any floor, prioritizing by remaining area
        for fl in sorted(floors, key=lambda x: assignments[x]['remaining_area'], reverse=True):
             if (assignments[fl]['remaining_area'] >= blk_area and
                assignments[fl]['remaining_capacity'] >= blk_capacity and
                can_place_block(blk, fl, assignments, mode)):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk.get('Department_Sub_Department', '').strip())
                cat = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip()
                if cat == 'ME':
                    assignments[fl]['ME_area'] += blk_area
                elif cat == 'WE':
                    assignments[fl]['WE_area'] += blk_area
                elif cat == 'US':
                    assignments[fl]['US_area'] += blk_area
                elif cat.lower() == 'support':
                    assignments[fl]['Support_area'] += blk_area
                elif cat.lower() == 'speciality':
                    assignments[fl]['Speciality_area'] += blk_area
                placed = True
                break
        if not placed:
            still_unassigned.append(blk)

    unassigned_blocks = still_unassigned # Update the list of unassigned blocks


    # Build output DataFrames
    rows=[]
    for fl,info in assignments.items():
        for blk in info['assigned_blocks']:
            rows.append({
                'Floor':fl,
                'Department':blk.get('Department_Sub_Department', ''), # Use .get for robustness
                'Block_Name':blk.get('Block_Name', ''), # Use .get for robustness
                'Destination_Group':blk.get('Destination_Group', ''), # Use .get for robustness
                'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''), # Use .get for robustness
                'Assigned_Area_SQM':blk.get('Cumulative_Area_SQM', 0), # Use .get for robustness
                'Max_Occupancy':blk.get('Max_Occupancy_with_Capacity', 0), # Use .get for robustness
                'Immovable-Movable Asset': blk.get('Immovable-Movable Asset', '') # Add this column
            })
    detailed_df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=['Floor', 'Department', 'Block_Name', 'Destination_Group', 'SpaceMix', 'Assigned_Area_SQM', 'Max_Occupancy', 'Immovable-Movable Asset'])

    summary = (
        detailed_df.groupby('Floor')
        .agg(Assgn_Blocks=('Block_Name','count'),
             Assgn_Area_SQM=('Assigned_Area_SQM','sum'),
             Total_Occupancy=('Max_Occupancy','sum'))
        .reset_index()
    ) if not detailed_df.empty else pd.DataFrame(columns=['Floor', 'Assgn_Blocks', 'Assgn_Area_SQM', 'Total_Occupancy'])

    input_sub = all_floor_data[['Name','Usable_Area_(SQM)','Max_Assignable_Floor_loading_Capacity']].rename(columns={'Name':'Floor','Usable_Area_(SQM)':'Input_Usable_Area_SQM','Max_Assignable_Floor_loading_Capacity':'Input_Max_Capacity'})
    floor_summary_df = pd.merge(input_sub, summary, on='Floor', how='left').fillna(0)

    # space-mix by %
    # Recalculate category totals based on the original all_block_data (renovation + existing)
    category_totals = {
        cat: len(all_block_data[
            all_block_data['SpaceMix_(ME_WE_US_Support_Speciality)'].astype(str).str.strip() == cat
        ])
        for cat in all_categories
    }

    mix_rows = [] # Renamed from rows to mix_rows to avoid confusion
    for fl, info in assignments.items():
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            cat = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip() # Use .get and strip for safety
            if cat in counts:
                counts[cat] += 1
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            cnt = counts[cat]
            # Percent of floor’s blocks
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            # Percent of overall blocks of that category
            total_cat = category_totals.get(cat, 0) # Use .get with default 0
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            mix_rows.append({ # Append to mix_rows
                'Floor': fl,
                'SpaceMix': cat,
                '%spaceMix': round(pct_overall, 2)

            })

    space_mix_df = pd.DataFrame(mix_rows) if mix_rows else pd.DataFrame(columns=['Floor', 'SpaceMix', '%spaceMix'])


    # 4.6.4 Unassigned DataFrame
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),
            'Block_Name': blk.get('Block_Name', ''),
            'Destination_Group': blk.get('Destination_Group', ''),
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0),
            'Immovable-Movable Asset': blk.get('Immovable-Movable Asset', '') # Add this column
        })
    unassigned_df = pd.DataFrame(unassigned_list) if unassigned_list else pd.DataFrame(columns=['Department', 'Block_Name', 'Destination_Group', 'SpaceMix', 'Area_SQM', 'Max_Occupancy', 'Immovable-Movable Asset'])


    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Generate & Export Excel + CSV Files (including Unassigned)
# ----------------------------------------

central_detailed, central_floor_sum, central_space_mix, central_unassigned = run_stack_plan('centralized')
semi_detailed,    semi_floor_sum,    semi_space_mix,    semi_unassigned    = run_stack_plan('semi')
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = run_stack_plan('decentralized')

# File names
central_file    = 'AAR1_stack_plan_centralized14.xlsx'
semi_file       = 'AAR1_stack_plan_semi_centralized14.xlsx'
decentral_file  = 'AAR1_stack_plan_decentralized14.xlsx'

# --- ExcelWriter blocks with an extra sheet "Unassigned" ---
with pd.ExcelWriter(central_file) as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(semi_file) as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(decentral_file) as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

print("✅ Generated three Excel outputs (each with an 'Unassigned' sheet):")
print(f"    • {central_file}")
print(f"    • {semi_file}")
print(f"    • {decentral_file}")

['Name', 'Usable_Area', 'Max_Assignable_Floor_loading_Capacity']
['Name', 'Usable_Area_(SQM)', 'Max_Assignable_Floor_loading_Capacity']
['Department_Sub-Department', 'Splittable', 'Min_%_of_Block']
['Department_Sub-Department', 'Splittable', 'Min_%of_Block_per_department']
✅ Generated three Excel outputs (each with an 'Unassigned' sheet):
    • AAR1_stack_plan_centralized14.xlsx
    • AAR1_stack_plan_semi_centralized14.xlsx
    • AAR1_stack_plan_decentralized14.xlsx


# AAR

In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load Input Sheets & Normalize
# ----------------------------------------

excel_path = '/content/AA-R2.xlsx'  # ← adjust if needed

# 1.1 Floors sheet
all_floor_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 2 - Floor'
)
all_floor_data.columns = all_floor_data.columns.str.strip()

# Normalize usable-area & capacity column names
floor_col_map = {}
for c in all_floor_data.columns:
    key = c.lower().replace(' ', '').replace('_','')
    if 'usable' in key and 'area' in key:
        floor_col_map[c] = 'Usable_Area'
    elif 'capacity' in key or 'loading' in key:
        floor_col_map[c] = 'Max_Assignable_Floor_loading_Capacity'
all_floor_data = all_floor_data.rename(columns=floor_col_map)

# 1.2 Blocks sheet
all_block_data = pd.read_excel(
    excel_path,
    sheet_name='RenovationProgram Table Input 1'
)
all_block_data.columns = all_block_data.columns.str.strip()

# —————————————————————————————————————————————
# Peel off Immovable Assets if those columns exist
# —————————————————————————————————————————————
if {'Immovable-Movable Asset', 'Level'}.issubset(all_block_data.columns):
    immovable_df = all_block_data.loc[
        all_block_data['Immovable-Movable Asset'].str.strip() == 'Immovable Asset'
    ].copy()
    immovable_df['Assigned_Floor'] = immovable_df['Level'].astype(str).str.strip()
    movable_blocks_df = all_block_data.drop(immovable_df.index).copy()
else:
    immovable_df = pd.DataFrame(columns=list(all_block_data.columns) + ['Assigned_Floor'])
    movable_blocks_df = all_block_data.copy()

# 1.3 Department Split sheet
department_split_data = pd.read_excel(
    excel_path,
    sheet_name='Department Split',
    skiprows=1
)
department_split_data.columns = department_split_data.columns.str.strip()
department_split_data = department_split_data.rename(
    columns={'BU_Department_Sub-Department': 'Department_Sub-Department'}
)

# 1.4 Adjacency sheet
xls = pd.ExcelFile(excel_path)
adjacency_sheet_name = [n for n in xls.sheet_names if "Adjacency" in n][0]
raw_adj = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_adj.apply(pd.to_numeric, errors='coerce')
adjacency_data.index = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()

# 1.5 De-Centralized Logic sheet
df_logic = pd.read_excel(
    excel_path,
    sheet_name='De-Centralized Logic',
    header=None
)
De_Centralized_data = {}
current = None
for _, r in df_logic.iterrows():
    cell = str(r[0]).strip() if pd.notna(r[0]) else ""
    if cell in ["Centralised", "Semi Centralized", "DeCentralised"]:
        current = cell
        De_Centralized_data[current] = {"Add": 0}
    elif current and cell == "( Add into cetralised destination Block)":
        De_Centralized_data[current]["Add"] = int(r[1]) if pd.notna(r[1]) else 0
for k in ["Centralised", "Semi Centralized", "DeCentralised"]:
    De_Centralized_data.setdefault(k, {"Add": 0})
# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------

# 2.1 Convert cumulative circulation area from SQFT → SQM
#all_block_data['Cumulative_Area_SQM'] = (
 #   all_block_data['Cumulative_Block_Circulation_Area_(SQM)']
#)

# Step 0: Assume all_block_data is already defined as your full DataFrame

# Step 1: Select Immovable Asset blocks
immovable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'].str.strip() == 'Immovable Asset'
].copy()

# Step 2: Select all other blocks (i.e., NOT immovable → movable or NA or others)
movable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'].str.strip() != 'Immovable Asset'
].copy()


# Step 2.1: From Movable blocks, select Destination or both
destination_blocks = movable_blocks[movable_blocks['Typical_Destination'].isin(['Destination', 'both'])].copy()

# Step 2.2: From Movable blocks, select Typical
typical_blocks = movable_blocks[movable_blocks['Typical_Destination'] == 'Typical'].copy()


# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Returns a dict keyed by floor name. Each entry tracks:
      - remaining_area
      - remaining_capacity
      - assigned_blocks      (list of block‐row dicts)
      - assigned_departments (set of sub‐departments)
      - ME_area, WE_area, US_area, Support_area, Speciality_area (floats)
    """
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

floors = list(initialize_floor_assignments(all_floor_data).keys())

# ----------------------------------------
# Step 4: Core Stacking Function
# ----------------------------------------

def run_stack_plan(mode):
    """
    mode: 'centralized', 'semi', or 'decentralized'
    Returns four DataFrames:
      1) detailed_df      – each block’s assigned floor, department, block name, destination group, space mix, area, occupancy
      2) floor_summary_df – floor‐wise totals (block count, total area, total occupancy)
      3) space_mix_df     – for each floor and each category {ME, WE, US, Support, Speciality}:
                              - Unit_Count_on_Floor
                              - Pct_of_Floor_UC      = (category_count_on_floor / total_blocks_on_floor) × 100%
                              - Pct_of_Overall_UC    = (category_count_on_floor / total_blocks_of_category_overall) × 100%
      4) unassigned_df    – blocks that couldn’t be placed
    """
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []

    import re
    # General approach to strip 'L' followed by 3 digits
    def clean_floor_name(floor_name):
        return re.sub(r'^L\d{3}', '', floor_name).strip()

    floor_name_map = {
        clean_floor_name(row['Name']): row['Name'].strip()
        for _, row in all_floor_data.iterrows()
    }


    # Assign immovable blocks based on 'Level'
    immovable_blocks = all_block_data[all_block_data['Immovable-Movable Asset'] == 'Immovable Asset'].copy()

    for _, block in immovable_blocks.iterrows():
        target_floor_raw = str(block['Level']).strip()
        target_floor = floor_name_map.get(target_floor_raw, None)

        block_area = block['Cumulative_Block_Circulation_Area']
        block_capacity = block['Max_Occupancy_with_Capacity']

        if target_floor in assignments:
            floor_data = assignments[target_floor]

            # Check area and capacity constraints
            if floor_data['remaining_area'] >= block_area and floor_data['remaining_capacity'] >= block_capacity:
                # Assign block
                floor_data['assigned_blocks'].append(block.to_dict())
                floor_data['assigned_departments'].add(block['Department_Sub_Department'])

                # Update remaining area and capacity
                floor_data['remaining_area'] -= block_area
                floor_data['remaining_capacity'] -= block_capacity

                # Update area category
                category = block['SpaceMix_(ME_WE_US_Support_Speciality)']
                category_key = f"{category}_area"
                if category_key in floor_data:
                    floor_data[category_key] += block_area
            else:
                unassigned_blocks.append(block.to_dict())
        else:
            unassigned_blocks.append(block.to_dict())


    # Determine how many floors to use for destination blocks
    def destination_floor_count():
        if mode == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    max_dest_floors = destination_floor_count()
    # Cap at total number of floors
    max_dest_floors = min(max_dest_floors, len(floors))

    # Pre‐compute each group's total area and total capacity
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Block_Circulation_Area']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Phase 1: Assign destination groups (try whole‐group first; if that fails, split across floors)
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        info_grp = dest_groups[grp]
        grp_area = info_grp['total_area']
        grp_cap  = info_grp['total_capacity']
        placed_whole = False

        # 4.2.a Attempt to place entire group on any of the first max_dest_floors
        candidate_floors = floors[:max_dest_floors].copy()

        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                # Entire group fits here—place all blocks
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(
                        blk['Department_Sub_Department']
                    )
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # 4.2.b If not yet placed, try the remaining floors (beyond max_dest_floors)
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(
                            blk['Department_Sub_Department'].strip()
                        )
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # 4.2.c If still not placed as a whole, split the group block‐by‐block across floors
        if not placed_whole:
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # Try placing group by removing the largest blocks one-by-one until remaining can be placed whole
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Block_Circulation_Area'], reverse=True)
                removed_blocks = []
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    trial_area = sum(b['Cumulative_Block_Circulation_Area'] for b in trial_blocks)
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)

                    # Try to place this reduced group
                    floor_combination = []
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    temp_success = True
                    for blk in trial_blocks:
                        blk_area = blk['Cumulative_Block_Circulation_Area']
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity):
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        # Apply final assignment for successfully placed trial blocks
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Block_Circulation_Area']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        # Remove one largest block and retry
                        removed_blocks.append(trial_blocks.pop(0))

                # Place removed blocks one-by-one
                for blk in removed_blocks:
                    blk_area = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= bll_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                # Even splitting won't fit all blocks, place block-by-block
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    blk_area     = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False

                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)

    # Phase 2A: Randomly assign ME blocks (typical)
    me_blocks = [
        blk for blk in typical_blocks.to_dict('records')
        if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
    ]
    random.shuffle(me_blocks)
    for blk in me_blocks:
        blk_area     = blk['Cumulative_Block_Circulation_Area']
        blk_capacity = blk['Max_Occupancy_with_Capacity']
        blk_dept     = blk['Department_Sub_Department'].strip()

        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        placed = False
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            print(f"Warning: Could not place ME block '{blk['Block_Name']}'")

    # Compute ME distribution per floor (unit counts)
    me_count_per_floor = {fl: 0 for fl in floors}
    for fl, info in assignments.items():
        me_count_per_floor[fl] = sum(
            1 for blk in info['assigned_blocks']
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
        )
    total_me = sum(me_count_per_floor.values())
    if total_me == 0:
        me_frac_per_floor = {fl: 1 / len(floors) for fl in floors}
    else:
        me_frac_per_floor = {fl: me_count_per_floor[fl] / total_me for fl in floors}

    # Phase 2B: Assign other categories proportionally
    other_categories = ['WE', 'US', 'Support', 'Speciality']
    for category in other_categories:
        cat_blocks = [
            blk for blk in typical_blocks.to_dict('records')
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == category
        ]
        total_cat = len(cat_blocks)
        if total_cat == 0:
            continue

        raw_targets = {fl: me_frac_per_floor[fl] * total_cat for fl in floors}
        target_counts = {fl: int(round(raw_targets[fl])) for fl in floors}

        diff = total_cat - sum(target_counts.values())
        if diff != 0:
            fractional_parts = {
                fl: raw_targets[fl] - math.floor(raw_targets[fl]) for fl in floors
            }
            if diff > 0:
                for fl in sorted(floors, key=lambda x: fractional_parts[x], reverse=True)[:diff]:
                    target_counts[fl] += 1
            else:
                for fl in sorted(floors, key=lambda x: fractional_parts[x])[: -diff]:
                    target_counts[fl] -= 1

        random.shuffle(cat_blocks)
        assigned_counts = {fl: 0 for fl in floors}

        for blk in cat_blocks:
            blk_area     = blk['Cumulative_Block_Circulation_Area']
            blk_capacity = blk['Max_Occupancy_with_Capacity']
            blk_dept     = blk['Department_Sub_Department'].strip()

            deficits = {fl: target_counts[fl] - assigned_counts[fl] for fl in floors}
            floors_with_deficit = [fl for fl, d in deficits.items() if d > 0]
            if floors_with_deficit:
                candidate_floors = sorted(
                    floors_with_deficit,
                    key=lambda x: deficits[x],
                    reverse=True
                )
            else:
                candidate_floors = floors.copy()

            placed = False
            for fl in candidate_floors:
                if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area'] -= blk_area
                    assignments[fl]['remaining_capacity'] -= blk_capacity
                    assignments[fl]['assigned_departments'].add(blk_dept)
                    if category == 'WE':
                        assignments[fl]['WE_area'] += blk_area
                    elif category == 'US':
                        assignments[fl]['US_area'] += blk_area
                    elif category == 'Support':
                        assignments[fl]['Support_area'] += blk_area
                    elif category == 'Speciality':
                        assignments[fl]['Speciality_area'] += blk_area
                    assigned_counts[fl] += 1
                    placed = True
                    break

            if not placed:
                fallback = floors.copy()
                random.shuffle(fallback)
                for fl in fallback:
                    if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['remaining_area'] -= blk_area
                        assignments[fl]['remaining_capacity'] -= blk_capacity
                        assignments[fl]['assigned_departments'].add(blk_dept)
                        if category == 'WE':
                            assignments[fl]['WE_area'] += blk_area
                        elif category == 'US':
                            assignments[fl]['US_area'] += blk_area
                        elif category == 'Support':
                            assignments[fl]['Support_area'] += blk_area
                        elif category == 'Speciality':
                            assignments[fl]['Speciality_area'] += blk_area
                        assigned_counts[fl] += 1
                        placed = True
                        break

            if not placed:
                print(f"Warning: Could not place {category} block '{blk['Block_Name']}'")
                unassigned_blocks.append(blk)
    # Re-attempt placing unassigned blocks on randomized floor order
    still_unassigned = []

    # Get list of floor names and shuffle
    floor_list = list(assignments.keys())
    random.shuffle(floor_list)  # This randomizes the order

    for block in unassigned_blocks:
        placed = False
        block_area = block['Cumulative_Block_Circulation_Area']
        block_capacity = block['Max_Occupancy_with_Capacity']
        department = block['Department_Sub_Department']
        category = block['SpaceMix_(ME_WE_US_Support_Speciality)']
        category_key = f"{category}_area"

        for floor in floor_list:
            data = assignments[floor]
            if data['remaining_area'] >= block_area and data['remaining_capacity'] >= block_capacity:
                # Assign the block to this floor
                data['assigned_blocks'].append(block)
                data['assigned_departments'].add(department)
                data['remaining_area'] -= block_area
                data['remaining_capacity'] -= block_capacity

                # Update category area
                if category_key in data:
                    data[category_key] += block_area

                placed = True
                break  # Move to next block

        if not placed:
            still_unassigned.append(block)

    # Update the global unassigned_blocks list
    unassigned_blocks = still_unassigned

    # Phase 3: Build Detailed & Summary DataFrames

    # 3.1 Detailed DataFrame
    assignment_list = []
    for fl, info in assignments.items():
        for blk in info['assigned_blocks']:
            assignment_list.append({
                'Block_id': blk['Block_ID'],
                'Floor': fl,
                'Department': blk['Department_Sub_Department'],
                'Block_Name': blk['Block_Name'],
                'Destination_Group': blk['Destination_Group'],
                'SpaceMix': blk['SpaceMix_(ME_WE_US_Support_Speciality)'],
                'Assigned_Area_SQM': blk['Cumulative_Block_Circulation_Area'],
                'Max_Occupancy': blk['Max_Occupancy_with_Capacity'],
                'Asset_Type': blk['Immovable-Movable Asset']  # <-- New column added
            })
    detailed_df = pd.DataFrame(assignment_list)

    # 4.6.2 Floor_Summary DataFrame
     # 3.2 “Floor_Summary” DataFrame
    floor_summary_df = (
    detailed_df
    .groupby('Floor')
    .agg(
        Assgn_Blocks=('Block_Name', 'count'),
        Assgn_Area_SQM=('Assigned_Area_SQM', 'sum'),
        Total_Occupancy=('Max_Occupancy', 'sum')
    )
    .reset_index()
)

    # Merge with original floor input data to get base values
    floor_input_subset = all_floor_data[[
    'Name', 'Usable_Area', 'Max_Assignable_Floor_loading_Capacity'
]].rename(columns={
    'Name': 'Floor',
    'Usable_Area': 'Input_Usable_Area',
    'Max_Assignable_Floor_loading_Capacity': 'Input_Max_Capacity'
})

    # Join input data with summary
    floor_summary_df = pd.merge(
    floor_input_subset,
    floor_summary_df,
    on='Floor',
    how='left'
)

    # Fill NaNs (if any floor didn't get any assignments)
    floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]] = floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]].fillna(0)
    # 3.3 SpaceMix_By_Units DataFrame
    all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']
    category_totals = {
        cat: len(typical_blocks[
            typical_blocks['SpaceMix_(ME_WE_US_Support_Speciality)'].str.strip() == cat
        ])
        for cat in all_categories
    }

    rows = []
    for fl, info in assignments.items():
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            cat = blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
            if cat in counts:
                counts[cat] += 1
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            cnt = counts[cat]
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            total_cat = category_totals[cat]
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            rows.append({
                'Floor': fl,
                'SpaceMix': cat,
                'Unit_Count_on_Floor': cnt,
                'Pct_of_Floor_UC': round(pct_of_floor, 2),
                'Pct_of_Overall_UC': round(pct_overall, 2)
            })

    space_mix_df = pd.DataFrame(rows)

     # 4.6.4 Unassigned DataFrame
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),
            'Block_Name': blk.get('Block_Name', ''),
            'Destination_Group': blk.get('Destination_Group', ''),
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0),
            'Asset_Type': blk['Immovable-Movable Asset']
        })
    unassigned_df = pd.DataFrame(unassigned_list)

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Generate & Export Three Excel Files
# ----------------------------------------
central_detailed, central_floor_sum, central_space_mix, central_unassigned = run_stack_plan('centralized')
semi_detailed, semi_floor_sum, semi_space_mix, semi_unassigned = run_stack_plan('semi')
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = run_stack_plan('decentralized')


with pd.ExcelWriter('stack_plan_centralized18.xlsx') as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


with pd.ExcelWriter('stack_plan_semi_centralized18.xlsx') as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


with pd.ExcelWriter('stack_plan_decentralized18.xlsx') as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


print("✅ Generated three Excel outputs:")
print("    • stack_plan_centralized8.xlsx")
print("    • stack_plan_semi_centralized8.xlsx")
print("    • stack_plan_decentralized8.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '/content/AA-R2.xlsx'

# AAR-2


In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load Input Sheets & Normalize
# ----------------------------------------

excel_path = '/content/AA- R2 (1).xlsx'  # ← adjust if needed

# 1.1 Floors sheet (skip first row)
all_floor_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 2 - Floor',
    skiprows=1  # skips the first row (0-indexed)
)

all_floor_data.columns = all_floor_data.columns.str.strip()

# Normalize usable-area & capacity column names
floor_col_map = {}
for c in all_floor_data.columns:
    key = c.lower().replace(' ', '').replace('_','')
    if 'usable' in key and 'area' in key:
        floor_col_map[c] = 'Usable_Area'
    elif 'capacity' in key or 'loading' in key:
        floor_col_map[c] = 'Max_Assignable_Floor_loading_Capacity'
all_floor_data = all_floor_data.rename(columns=floor_col_map)

# 1.2 Blocks sheet
all_block_data = pd.read_excel(
    excel_path,
    sheet_name='Renovation Program Table Input '
)
all_block_data.columns = all_block_data.columns.str.strip()

# —————————————————————————————————————————————
# Peel off Immovable Assets if those columns exist
# —————————————————————————————————————————————
if {'Immovable-Movable Asset', 'Level'}.issubset(all_block_data.columns):
    immovable_df = all_block_data.loc[
        all_block_data['Immovable-Movable Asset'].str.strip() == 'Immovable Asset'
    ].copy()
    immovable_df['Assigned_Floor'] = immovable_df['Level'].astype(str).str.strip()
    movable_blocks_df = all_block_data.drop(immovable_df.index).copy()
else:
    immovable_df = pd.DataFrame(columns=list(all_block_data.columns) + ['Assigned_Floor'])
    movable_blocks_df = all_block_data.copy()

# 1.3 Department Split sheet
department_split_data = pd.read_excel(
    excel_path,
    sheet_name='Department Split',
    skiprows=1
)
department_split_data.columns = department_split_data.columns.str.strip()
department_split_data = department_split_data.rename(
    columns={'BU_Department_Sub-Department': 'Department_Sub-Department'}
)

# 1.4 Adjacency sheet
xls = pd.ExcelFile(excel_path)
adjacency_sheet_name = [n for n in xls.sheet_names if "Adjacency" in n][0]
raw_adj = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_adj.apply(pd.to_numeric, errors='coerce')
adjacency_data.index = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()

# 1.5 De-Centralized Logic sheet
df_logic = pd.read_excel(
    excel_path,
    sheet_name='De-Centralized Logic',
    header=None
)
De_Centralized_data = {}
current = None
for _, r in df_logic.iterrows():
    cell = str(r[0]).strip() if pd.notna(r[0]) else ""
    if cell in ["Centralised", "Semi Centralized", "DeCentralised"]:
        current = cell
        De_Centralized_data[current] = {"Add": 0}
    elif current and cell == "( Add into cetralised destination Block)":
        De_Centralized_data[current]["Add"] = int(r[1]) if pd.notna(r[1]) else 0
for k in ["Centralised", "Semi Centralized", "DeCentralised"]:
    De_Centralized_data.setdefault(k, {"Add": 0})
# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------

# 2.1 Convert cumulative circulation area from SQFT → SQM
#all_block_data['Cumulative_Area_SQM'] = (
 #   all_block_data['Cumulative_Block_Circulation_Area_(SQM)']
#)

# Step 0: Assume all_block_data is already defined as your full DataFrame

# Step 1: Select Immovable Asset blocks
immovable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'].str.strip() == 'Immovable Asset'
].copy()

# Step 2: Select all other blocks (i.e., NOT immovable → movable or NA or others)
movable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'].str.strip() != 'Immovable Asset'
].copy()


# Step 2.1: From Movable blocks, select Destination or both
destination_blocks = movable_blocks[movable_blocks['Typical_Destination'].isin(['Destination', 'both'])].copy()

# Step 2.2: From Movable blocks, select Typical
typical_blocks = movable_blocks[movable_blocks['Typical_Destination'] == 'Typical'].copy()


# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Returns a dict keyed by floor name. Each entry tracks:
      - remaining_area
      - remaining_capacity
      - assigned_blocks      (list of block‐row dicts)
      - assigned_departments (set of sub‐departments)
      - ME_area, WE_area, US_area, Support_area, Speciality_area (floats)
    """
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

floors = list(initialize_floor_assignments(all_floor_data).keys())

# ----------------------------------------
# Step 4: Core Stacking Function
# ----------------------------------------

def run_stack_plan(mode):
    """
    mode: 'centralized', 'semi', or 'decentralized'
    Returns four DataFrames:
      1) detailed_df      – each block’s assigned floor, department, block name, destination group, space mix, area, occupancy
      2) floor_summary_df – floor‐wise totals (block count, total area, total occupancy)
      3) space_mix_df     – for each floor and each category {ME, WE, US, Support, Speciality}:
                              - Unit_Count_on_Floor
                              - Pct_of_Floor_UC      = (category_count_on_floor / total_blocks_on_floor) × 100%
                              - Pct_of_Overall_UC    = (category_count_on_floor / total_blocks_of_category_overall) × 100%
      4) unassigned_df    – blocks that couldn’t be placed
    """
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []

    import re

    def normalize_floor_name(name):
        """Standardize floor names like 'L002Floor 01' or '3 Floor' → 'Floor 03'"""
        name = str(name).strip()

        # Remove prefix like L001
        name = re.sub(r'^L\d{3}', '', name).strip()

        # If format is like '3 Floor' → convert to 'Floor 03'
        match = re.match(r'^(\d+)\s+Floor$', name)
        if match:
            number = int(match.group(1))
            return f"Floor {number:02d}"

        return name  # for 'Ground Floor', 'Floor 01', etc.
      # Map normalized name → actual floor name in assignments
    floor_name_map = {
        normalize_floor_name(row['Name']): row['Name'].strip()
        for _, row in all_floor_data.iterrows()
    }





    # Assign immovable blocks based on 'Level'
    immovable_blocks = all_block_data[all_block_data['Immovable-Movable Asset'] == 'Immovable Asset'].copy()

    for _, block in immovable_blocks.iterrows():
        target_floor_raw = str(block['Level']).strip()
        target_floor = floor_name_map.get(target_floor_raw, None)

        block_area = block['Cumulative_Block_Circulation_Area']
        block_capacity = block['Max_Occupancy_with_Capacity']

        if target_floor in assignments:
            floor_data = assignments[target_floor]

            # Check area and capacity constraints
            if floor_data['remaining_area'] >= block_area and floor_data['remaining_capacity'] >= block_capacity:
                # Assign block
                floor_data['assigned_blocks'].append(block.to_dict())
                floor_data['assigned_departments'].add(block['Department_Sub_Department'])

                # Update remaining area and capacity
                floor_data['remaining_area'] -= block_area
                floor_data['remaining_capacity'] -= block_capacity

                # Update area category
                category = block['SpaceMix_(ME_WE_US_Support_Speciality)']
                category_key = f"{category}_area"
                if category_key in floor_data:
                    floor_data[category_key] += block_area
            else:
                unassigned_blocks.append(block.to_dict())
        else:
            unassigned_blocks.append(block.to_dict())


    # Determine how many floors to use for destination blocks
    def destination_floor_count():
        if mode == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    max_dest_floors = destination_floor_count()
    # Cap at total number of floors
    max_dest_floors = min(max_dest_floors, len(floors))

    # Pre‐compute each group's total area and total capacity
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Block_Circulation_Area']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Phase 1: Assign destination groups (try whole‐group first; if that fails, split across floors)
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        info_grp = dest_groups[grp]
        grp_area = info_grp['total_area']
        grp_cap  = info_grp['total_capacity']
        placed_whole = False

        # 4.2.a Attempt to place entire group on any of the first max_dest_floors
        candidate_floors = floors[:max_dest_floors].copy()

        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                # Entire group fits here—place all blocks
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(
                        blk['Department_Sub_Department']
                    )
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # 4.2.b If not yet placed, try the remaining floors (beyond max_dest_floors)
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(
                            blk['Department_Sub_Department'].strip()
                        )
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # 4.2.c If still not placed as a whole, split the group block‐by‐block across floors
        if not placed_whole:
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # Try placing group by removing the largest blocks one-by-one until remaining can be placed whole
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Block_Circulation_Area'], reverse=True)
                removed_blocks = []
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    trial_area = sum(b['Cumulative_Block_Circulation_Area'] for b in trial_blocks)
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)

                    # Try to place this reduced group
                    floor_combination = []
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    temp_success = True
                    for blk in trial_blocks:
                        blk_area = blk['Cumulative_Block_Circulation_Area']
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity):
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        # Apply final assignment for successfully placed trial blocks
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Block_Circulation_Area']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        # Remove one largest block and retry
                        removed_blocks.append(trial_blocks.pop(0))

                # Place removed blocks one-by-one
                for blk in removed_blocks:
                    blk_area = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= bll_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                # Even splitting won't fit all blocks, place block-by-block
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    blk_area     = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False

                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity):
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)

    # Phase 2A: Randomly assign ME blocks (typical)
    me_blocks = [
        blk for blk in typical_blocks.to_dict('records')
        if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
    ]
    random.shuffle(me_blocks)
    for blk in me_blocks:
        blk_area     = blk['Cumulative_Block_Circulation_Area']
        blk_capacity = blk['Max_Occupancy_with_Capacity']
        blk_dept     = blk['Department_Sub_Department'].strip()

        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        placed = False
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            print(f"Warning: Could not place ME block '{blk['Block_Name']}'")

    # Compute ME distribution per floor (unit counts)
    me_count_per_floor = {fl: 0 for fl in floors}
    for fl, info in assignments.items():
        me_count_per_floor[fl] = sum(
            1 for blk in info['assigned_blocks']
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == 'ME'
        )
    total_me = sum(me_count_per_floor.values())
    if total_me == 0:
        me_frac_per_floor = {fl: 1 / len(floors) for fl in floors}
    else:
        me_frac_per_floor = {fl: me_count_per_floor[fl] / total_me for fl in floors}

    # Phase 2B: Assign other categories proportionally
    other_categories = ['WE', 'US', 'Support', 'Speciality']
    for category in other_categories:
        cat_blocks = [
            blk for blk in typical_blocks.to_dict('records')
            if blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip() == category
        ]
        total_cat = len(cat_blocks)
        if total_cat == 0:
            continue

        raw_targets = {fl: me_frac_per_floor[fl] * total_cat for fl in floors}
        target_counts = {fl: int(round(raw_targets[fl])) for fl in floors}

        diff = total_cat - sum(target_counts.values())
        if diff != 0:
            fractional_parts = {
                fl: raw_targets[fl] - math.floor(raw_targets[fl]) for fl in floors
            }
            if diff > 0:
                for fl in sorted(floors, key=lambda x: fractional_parts[x], reverse=True)[:diff]:
                    target_counts[fl] += 1
            else:
                for fl in sorted(floors, key=lambda x: fractional_parts[x])[: -diff]:
                    target_counts[fl] -= 1

        random.shuffle(cat_blocks)
        assigned_counts = {fl: 0 for fl in floors}

        for blk in cat_blocks:
            blk_area     = blk['Cumulative_Block_Circulation_Area']
            blk_capacity = blk['Max_Occupancy_with_Capacity']
            blk_dept     = blk['Department_Sub_Department'].strip()

            deficits = {fl: target_counts[fl] - assigned_counts[fl] for fl in floors}
            floors_with_deficit = [fl for fl, d in deficits.items() if d > 0]
            if floors_with_deficit:
                candidate_floors = sorted(
                    floors_with_deficit,
                    key=lambda x: deficits[x],
                    reverse=True
                )
            else:
                candidate_floors = floors.copy()

            placed = False
            for fl in candidate_floors:
                if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area'] -= blk_area
                    assignments[fl]['remaining_capacity'] -= blk_capacity
                    assignments[fl]['assigned_departments'].add(blk_dept)
                    if category == 'WE':
                        assignments[fl]['WE_area'] += blk_area
                    elif category == 'US':
                        assignments[fl]['US_area'] += blk_area
                    elif category == 'Support':
                        assignments[fl]['Support_area'] += blk_area
                    elif category == 'Speciality':
                        assignments[fl]['Speciality_area'] += blk_area
                    assigned_counts[fl] += 1
                    placed = True
                    break

            if not placed:
                fallback = floors.copy()
                random.shuffle(fallback)
                for fl in fallback:
                    if (assignments[fl]['remaining_area'] >= blk_area and assignments[fl]['remaining_capacity'] >= blk_capacity):
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['remaining_area'] -= blk_area
                        assignments[fl]['remaining_capacity'] -= blk_capacity
                        assignments[fl]['assigned_departments'].add(blk_dept)
                        if category == 'WE':
                            assignments[fl]['WE_area'] += blk_area
                        elif category == 'US':
                            assignments[fl]['US_area'] += blk_area
                        elif category == 'Support':
                            assignments[fl]['Support_area'] += blk_area
                        elif category == 'Speciality':
                            assignments[fl]['Speciality_area'] += blk_area
                        assigned_counts[fl] += 1
                        placed = True
                        break

            if not placed:
                print(f"Warning: Could not place {category} block '{blk['Block_Name']}'")
                unassigned_blocks.append(blk)
    # Re-attempt placing unassigned blocks on randomized floor order
    still_unassigned = []

    # Get list of floor names and shuffle
    floor_list = list(assignments.keys())
    random.shuffle(floor_list)  # This randomizes the order

    for block in unassigned_blocks:
        placed = False
        block_area = block['Cumulative_Block_Circulation_Area']
        block_capacity = block['Max_Occupancy_with_Capacity']
        department = block['Department_Sub_Department']
        category = block['SpaceMix_(ME_WE_US_Support_Speciality)']
        category_key = f"{category}_area"

        for floor in floor_list:
            data = assignments[floor]
            if data['remaining_area'] >= block_area and data['remaining_capacity'] >= block_capacity:
                # Assign the block to this floor
                data['assigned_blocks'].append(block)
                data['assigned_departments'].add(department)
                data['remaining_area'] -= block_area
                data['remaining_capacity'] -= block_capacity

                # Update category area
                if category_key in data:
                    data[category_key] += block_area

                placed = True
                break  # Move to next block

        if not placed:
            still_unassigned.append(block)

    # Update the global unassigned_blocks list
    unassigned_blocks = still_unassigned

    # Phase 3: Build Detailed & Summary DataFrames

    # 3.1 Detailed DataFrame
    assignment_list = []
    for fl, info in assignments.items():
        for blk in info['assigned_blocks']:
            assignment_list.append({
                'Block_id': blk['Block_ID'],
                'Floor': fl,
                'Department': blk['Department_Sub_Department'],
                'Block_Name': blk['Block_Name'],
                'Destination_Group': blk['Destination_Group'],
                'SpaceMix': blk['SpaceMix_(ME_WE_US_Support_Speciality)'],
                'Assigned_Area_SQM': blk['Cumulative_Block_Circulation_Area'],
                'Max_Occupancy': blk['Max_Occupancy_with_Capacity'],
                'Asset_Type': blk['Immovable-Movable Asset']  # <-- New column added
            })
    detailed_df = pd.DataFrame(assignment_list)

    # 4.6.2 Floor_Summary DataFrame
     # 3.2 “Floor_Summary” DataFrame
    floor_summary_df = (
    detailed_df
    .groupby('Floor')
    .agg(
        Assgn_Blocks=('Block_Name', 'count'),
        Assgn_Area_SQM=('Assigned_Area_SQM', 'sum'),
        Total_Occupancy=('Max_Occupancy', 'sum')
    )
    .reset_index()
)

    # Merge with original floor input data to get base values
    floor_input_subset = all_floor_data[[
    'Name', 'Usable_Area', 'Max_Assignable_Floor_loading_Capacity'
]].rename(columns={
    'Name': 'Floor',
    'Usable_Area': 'Input_Usable_Area',
    'Max_Assignable_Floor_loading_Capacity': 'Input_Max_Capacity'
})

    # Join input data with summary
    floor_summary_df = pd.merge(
    floor_input_subset,
    floor_summary_df,
    on='Floor',
    how='left'
)

    # Fill NaNs (if any floor didn't get any assignments)
    floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]] = floor_summary_df[[
    'Assgn_Blocks',
    'Assgn_Area_SQM',
    'Total_Occupancy'
]].fillna(0)
    # 3.3 SpaceMix_By_Units DataFrame
    all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']
    category_totals = {
        cat: len(typical_blocks[
            typical_blocks['SpaceMix_(ME_WE_US_Support_Speciality)'].str.strip() == cat
        ])
        for cat in all_categories
    }

    rows = []
    for fl, info in assignments.items():
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            cat = blk['SpaceMix_(ME_WE_US_Support_Speciality)'].strip()
            if cat in counts:
                counts[cat] += 1
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            cnt = counts[cat]
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            total_cat = category_totals[cat]
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            rows.append({
                'Floor': fl,
                'SpaceMix': cat,
                'Unit_Count_on_Floor': cnt,
                'Pct_of_Floor_UC': round(pct_of_floor, 2),
                'Pct_of_Overall_UC': round(pct_overall, 2)
            })

    space_mix_df = pd.DataFrame(rows)

     # 4.6.4 Unassigned DataFrame
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),
            'Block_Name': blk.get('Block_Name', ''),
            'Destination_Group': blk.get('Destination_Group', ''),
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0),
            'Asset_Type': blk['Immovable-Movable Asset']
        })
    unassigned_df = pd.DataFrame(unassigned_list)

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Generate & Export Three Excel Files
# ----------------------------------------
central_detailed, central_floor_sum, central_space_mix, central_unassigned = run_stack_plan('centralized')
semi_detailed, semi_floor_sum, semi_space_mix, semi_unassigned = run_stack_plan('semi')
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = run_stack_plan('decentralized')


with pd.ExcelWriter('stack_plan_centralized20.xlsx') as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


with pd.ExcelWriter('stack_plan_semi_centralized20.xlsx') as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


with pd.ExcelWriter('stack_plan_decentralized20.xlsx') as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)


print("✅ Generated three Excel outputs:")
print("    • stack_plan_centralized8.xlsx")
print("    • stack_plan_semi_centralized8.xlsx")
print("    • stack_plan_decentralized8.xlsx")

✅ Generated three Excel outputs:
    • stack_plan_centralized8.xlsx
    • stack_plan_semi_centralized8.xlsx
    • stack_plan_decentralized8.xlsx


# BAR

In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load Input Sheets & Normalize
# ----------------------------------------
excel_path = '/content/BAR-2.xlsx'  # ← adjust if needed

# Floors sheet reader + normalization
def normalize_floor_cols(df):
    mapping = {}
    for c in df.columns:
        key = c.lower().replace(' ', '').replace('_','')
        if 'usable' in key and 'area' in key:
            mapping[c] = 'Usable_Area'
        elif 'capacity' in key or 'loading' in key:
            mapping[c] = 'Max_Assignable_Floor_loading_Capacity'
    return df.rename(columns=mapping)

all_floor_data = pd.read_excel(excel_path, sheet_name='Program Table Input 2 - Floor')
all_floor_data.columns = all_floor_data.columns.str.strip()
all_floor_data = normalize_floor_cols(all_floor_data)

# Blocks sheet
all_block_data = pd.read_excel(excel_path, sheet_name='Existing Program Table Input 1.')
all_block_data.columns = all_block_data.columns.str.strip()

# Department Split + splittable map
department_split_data = pd.read_excel(excel_path, sheet_name='Department Split', skiprows=1)
department_split_data.columns = department_split_data.columns.str.strip()
department_split_data = department_split_data.rename(columns={'BU_Department_Sub-Department': 'Department_Sub-Department'})
split_map = department_split_data.set_index('Department_Sub-Department')['Splittable'].to_dict() if 'Splittable' in department_split_data.columns else {}

# De-centralized logic sheet
logic_df = pd.read_excel(excel_path, sheet_name='De-Centralized Logic', header=None)
De_Centralized_data = {}
current = None
for _, r in logic_df.iterrows():
    label = str(r[0]).strip() if pd.notna(r[0]) else ''
    if label in ['Centralised', 'Semi Centralized', 'DeCentralised']:
        current = label
        De_Centralized_data[current] = {'Add': 0}
    elif current and 'Add into' in label:
        De_Centralized_data[current]['Add'] = int(r[1]) if pd.notna(r[1]) else 0
for key in ['Centralised', 'Semi Centralized', 'DeCentralised']:
    De_Centralized_data.setdefault(key, {'Add': 0})

# Preprocess blocks
immovable_blocks = all_block_data[all_block_data['Immovable-Movable Asset'].str.strip() == 'Immovable Asset']
movable_blocks   = all_block_data[all_block_data['Immovable-Movable Asset'].str.strip() != 'Immovable Asset']
destination_blocks = movable_blocks[movable_blocks['Typical_Destination'].isin(['Destination','both'])]
typical_blocks     = movable_blocks[movable_blocks['Typical_Destination'] == 'Typical']

# Initialize floor assignments
def initialize_floor_assignments(df):
    assigns = {}
    for _, row in df.iterrows():
        floor = row['Name'].strip()
        assigns[floor] = {
            'remaining_area': row['Usable_Area'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assigns

floors = list(initialize_floor_assignments(all_floor_data).keys())

# Core assignment function
def run_stack_plan(mode):
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []
    import re
    clean_name = lambda x: re.sub(r'^L\d{3}', '', x).strip()
    floor_map = {clean_name(r['Name']): r['Name'].strip() for _,r in all_floor_data.iterrows()}

    # Assign immovable blocks
    for _, blk in immovable_blocks.iterrows():
        lvl = str(blk['Level']).strip()
        fl = floor_map.get(lvl)
        area = blk.get('Cumulative_Block_Circulation_Area_(SQM)', blk.get('Cumulative_Block_Circulation_Area', 0))
        cap = blk.get('Max_Occupancy_with_Capacity', 0)
        if fl and assignments[fl]['remaining_area'] >= area and assignments[fl]['remaining_capacity'] >= cap:
            assignments[fl]['assigned_blocks'].append(blk.to_dict())
            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'])
            assignments[fl]['remaining_area'] -= area
            assignments[fl]['remaining_capacity'] -= cap
        else:
            unassigned_blocks.append(blk.to_dict())

    # Assign destination blocks
    def dest_count():
        if mode == 'centralized': return 2
        key = 'Semi Centralized' if mode == 'semi' else 'DeCentralised'
        return 2 + De_Centralized_data.get(key, {}).get('Add', 0)
    max_dest = min(dest_count(), len(floors))
    groups = {}
    for _, b in destination_blocks.iterrows(): groups.setdefault(b['Destination_Group'], []).append(b.to_dict())
    for blks in groups.values():
        total_area = sum(x.get('Cumulative_Block_Circulation_Area_(SQM)', x.get('Cumulative_Block_Circulation_Area',0)) for x in blks)
        placed = False
        for fl in floors[:max_dest]:
            if assignments[fl]['remaining_area'] >= total_area:
                for x in blks:
                    assignments[fl]['assigned_blocks'].append(x)
                    assignments[fl]['assigned_departments'].add(x['Department_Sub_Department'])
                assignments[fl]['remaining_area'] -= total_area
                placed = True
                break
        if not placed:
            for x in blks:
                a = x.get('Cumulative_Block_Circulation_Area_(SQM)', x.get('Cumulative_Block_Circulation_Area',0))
                for fl in sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True):
                    if assignments[fl]['remaining_area'] >= a:
                        assignments[fl]['assigned_blocks'].append(x)
                        assignments[fl]['assigned_departments'].add(x['Department_Sub_Department'])
                        assignments[fl]['remaining_area'] -= a
                        break
                else:
                    unassigned_blocks.append(x)

    # Assign typical blocks
    for blk in typical_blocks.to_dict('records'):
        dept = blk['Department_Sub_Department'].strip()
        spl = split_map.get(dept, -1)
        area = blk.get('Cumulative_Block_Circulation_Area_(SQM)', blk.get('Cumulative_Block_Circulation_Area',0))
        placed=False
        targets = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True) if spl != -1 else random.sample(floors, len(floors))
        for fl in targets:
            if assignments[fl]['remaining_area'] >= area:
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['assigned_departments'].add(dept)
                assignments[fl]['remaining_area'] -= area
                placed=True
                break
        if not placed: unassigned_blocks.append(blk)

    return assignments, unassigned_blocks

# Output builder
def build_outputs(assignments, unassigned_blocks):
    # Detailed
    detailed=[]
    for fl,info in assignments.items():
        for b in info['assigned_blocks']:
            detailed.append({
                'Block_ID':b.get('Block_ID'), 'Floor':fl,
                'Department':b.get('Department_Sub-Department'),'Block_Name':b.get('Block_Name'),
                'Destination_Group':b.get('Destination_Group'),'SpaceMix':b.get('SpaceMix_(ME_WE_US_Support_Speciality)'),
                'Assigned_Area_SQM':b.get('Cumulative_Block_Circulation_Area_(SQM)', b.get('Cumulative_Block_Circulation_Area',0)),
                'Max_Occupancy':b.get('Max_Occupancy_with_Capacity'),'Asset_Type':b.get('Immovable-Movable Asset')
            })
    df_det = pd.DataFrame(detailed)
    # Summary with allowed values
    df_sum = (df_det.groupby('Floor')
              .agg(Assgn_Blocks=('Block_Name','count'), Assgn_Area=('Assigned_Area_SQM','sum'), Total_Occ=('Max_Occupancy','sum'))
              .reset_index())
    # merge allowed usable area and capacity
    floor_allowed = all_floor_data[['Name','Usable_Area','Max_Assignable_Floor_loading_Capacity']].rename(columns={'Name':'Floor',
                                                                                                'Usable_Area':'Allowed_Usable_Area',
                                                                                                'Max_Assignable_Floor_loading_Capacity':'Allowed_Max_Occupancy'})
    df_sum = pd.merge(floor_allowed, df_sum, on='Floor', how='left').fillna({'Assgn_Blocks':0,'Assgn_Area':0,'Total_Occ':0})
    # SpaceMix
    cats=['ME','WE','US','Support','Speciality']; totals={c:len(df_det[df_det['SpaceMix']==c]) for c in cats}
    rows=[]
    for fl in df_sum['Floor']:
        sub=df_det[df_det['Floor']==fl]; tot=len(sub)
        for c in cats:
            cnt=len(sub[sub['SpaceMix']==c])
            rows.append({'Floor':fl,'SpaceMix':c,'Unit_Count_on_Floor':cnt,
                         'Pct_of_Floor_UC':round(cnt/tot*100,2) if tot else 0,
                         'Pct_of_Overall_UC':round(cnt/totals.get(c,1)*100,2)})
    df_space = pd.DataFrame(rows)
    # Unassigned
    ua=[]
    for b in unassigned_blocks:
        ua.append({'Department':b.get('Department_Sub-Department'),'Block_Name':b.get('Block_Name'),
                   'Destination_Group':b.get('Destination_Group'),'SpaceMix':b.get('SpaceMix_(ME_WE_US_Support_Speciality)'),
                   'Area_SQM':b.get('Cumulative_Block_Circulation_Area_(SQM)', b.get('Cumulative_Block_Circulation_Area',0)),
                   'Max_Occupancy':b.get('Max_Occupancy_with_Capacity'),'Asset_Type':b.get('Immovable-Movable Asset')})
    df_un = pd.DataFrame(ua)
    return df_det, df_sum, df_space, df_un

# ----------------------------------------
# Step 7: Execute & Export Three Workbooks
# ----------------------------------------
modes = [('centralized','stack_plan_centralized.xlsx'),
         ('semi','stack_plan_semi_centralized.xlsx'),
         ('decentralized','stack_plan_decentralized.xlsx')]
for mode, fname in modes:
    assigns, unassigned = run_stack_plan(mode)
    det, summ, space, unass = build_outputs(assigns, unassigned)
    with pd.ExcelWriter(fname) as writer:
        det.to_excel(writer, sheet_name='Detailed', index=False)
        summ.to_excel(writer, sheet_name='Floor_Summary', index=False)
        space.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
        unass.to_excel(writer, sheet_name='Unassigned', index=False)
print("✅ Generated three Excel files with allowed values: central, semi, and decentralized plans.")

✅ Generated three Excel files with allowed values: central, semi, and decentralized plans.


# CR-1,2

In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load Input Sheets from C-R1.xlsx
# ----------------------------------------
excel_path = '/content/C-R1.xlsx'  # adjust as needed

# 1.1 Floors sheet
all_floor_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 2 - Floor'
)
all_floor_data.columns = all_floor_data.columns.str.strip()
all_floor_data = all_floor_data.rename(columns={
    all_floor_data.columns[0]: 'Name',
    all_floor_data.columns[1]: 'Usable_Area_SQM',
    all_floor_data.columns[2]: 'Max_Capacity'
})
all_floor_data['Usable_Area_SQM'] = pd.to_numeric(all_floor_data['Usable_Area_SQM'], errors='raise')
all_floor_data['Max_Capacity']   = pd.to_numeric(all_floor_data['Max_Capacity'],   errors='raise')

# 1.2 Blocks sheet: normalize and map
blk = pd.read_excel(excel_path, sheet_name='Program Table Input 1 - Block')
blk.columns = (
    blk.columns
       .str.strip()
       .str.lower()
       .str.replace(r'[^\w]+', '_', regex=True)
       .str.strip('_')
)
# detect key columns dynamically
cum_col = next(c for c in blk.columns if 'cumulative' in c and 'circulation' in c)
occ_col = next(c for c in blk.columns if 'max' in c and 'capacity' in c)
td_col  = next(c for c in blk.columns if 'typical' in c and 'destination' in c)
hn_col  = next(c for c in blk.columns if 'neighborhood' in c)
sm_col  = next(c for c in blk.columns if 'spacemix' in c)
all_block_data = blk.copy()
all_block_data['cumulative_area_sqm'] = pd.to_numeric(all_block_data[cum_col], errors='coerce')
all_block_data['max_occupancy']       = pd.to_numeric(all_block_data[occ_col], errors='coerce')
all_block_data['typical_destination'] = all_block_data[td_col].astype(str).str.strip()
all_block_data['neighborhood']        = all_block_data[hn_col].astype(str).str.strip()
all_block_data['spacemix']            = all_block_data[sm_col].astype(str).str.strip()
# ensure level columns exist
for lvl in ['level_1','level_2','level_3','level_1_area','level_2_area','level_3_area']:
    if lvl not in all_block_data.columns:
        all_block_data[lvl] = None

# 1.3 Department Split sheet: dynamic header
dept_df = pd.read_excel(excel_path, sheet_name='Department Split', header=None)
hdr = dept_df.iloc[0].fillna('').astype(str).str.strip().tolist()
dept_df.columns = hdr
dept_df = dept_df.iloc[1:].reset_index(drop=True)
dep_col = next(c for c in hdr if 'department' in c.lower() and 'sub' in c.lower())
spl_col = next(c for c in hdr if 'splittable' in c.lower())
min_col = next(c for c in hdr if '%' in c and 'min' in c.lower())
dept_splittable = dept_df.set_index(dep_col)[spl_col].astype(int).to_dict()
dept_min_pct    = dept_df.set_index(dep_col)[min_col].astype(float).to_dict()

# 1.4 Adjacency sheets
xls = pd.ExcelFile(excel_path)
adj_sheets = [s for s in xls.sheet_names if 'Adjacency' in s]
adj_sub_df = pd.read_excel(excel_path,
                           sheet_name=[s for s in adj_sheets if 'Neighborhood' not in s][0],
                           header=1, index_col=0)
adj_sub_df = adj_sub_df.apply(pd.to_numeric, errors='coerce')
adj_nh_df = None
nh_sheets = [s for s in adj_sheets if 'Neighborhood' in s]
if nh_sheets:
    adj_nh_df = pd.read_excel(excel_path, sheet_name=nh_sheets[0], header=1, index_col=0)
    adj_nh_df = adj_nh_df.apply(pd.to_numeric, errors='coerce')

# 1.5 De-Centralized Logic sheet
logic_df = pd.read_excel(excel_path, sheet_name='De-Centralized Logic', header=None)
DeC_data, current = {}, None
for _, row in logic_df.iterrows():
    key = str(row[0]).strip()
    if key in ['Centralised','Semi Centralized','DeCentralised']:
        current = key
        DeC_data[current] = {'Add': 0}
    elif current and '( Add' in key:
        DeC_data[current]['Add'] = int(row[1]) if pd.notna(row[1]) else 0
for k in ['Centralised','Semi Centralized','DeCentralised']:
    DeC_data.setdefault(k, {'Add': 0})

# ----------------------------------------
# Step 2: Split Destination vs. Typical
# ----------------------------------------
dest_blocks = all_block_data[all_block_data['typical_destination'] == 'Destination']
typ_blocks  = all_block_data[all_block_data['typical_destination'] == 'Typical']

# Helper: initialize floors

def init_floors():
    floors_dict = {}
    for _, r in all_floor_data.iterrows():
        fl = r['Name'].strip()
        floors_dict[fl] = {
            'remaining_area': r['Usable_Area_SQM'],
            'remaining_capacity': r['Max_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return floors_dict

floors = list(init_floors().keys())

def dest_floor_count(mode):
    if mode == 'centralized':
        return 2
    if mode == 'semi':
        return 2 + DeC_data['Semi Centralized']['Add']
    if mode == 'decentralized':
        return 2 + DeC_data['DeCentralised']['Add']
    return 2

# Main stacking function

def run_stack_plan(mode):
    assignments = init_floors()
    unassigned = []

    # Phase 1: Destination groups
    grp_info = {}
    for _, blk in dest_blocks.iterrows():
        grp = blk['destination_group']
        grp_info.setdefault(grp, {'blocks': [], 'area': 0, 'cap': 0})
        d = blk.to_dict()
        grp_info[grp]['blocks'].append(d)
        grp_info[grp]['area'] += d['cumulative_area_sqm']
        grp_info[grp]['cap'] += d['max_occupancy']

    limit = min(dest_floor_count(mode), len(floors))
    for info in grp_info.values():
        placed = False
        # try primary floors
        for fl in floors[:limit]:
            if (assignments[fl]['remaining_area'] >= info['area'] and
                assignments[fl]['remaining_capacity'] >= info['cap']):
                for b in info['blocks']:
                    assignments[fl]['assigned_blocks'].append(b)
                    assignments[fl]['remaining_area'] -= b['cumulative_area_sqm']
                    assignments[fl]['remaining_capacity'] -= b['max_occupancy']
                    assignments[fl]['assigned_departments'].add(b['department_sub_department'].strip())
                placed = True
                break
        # try other floors
        if not placed:
            for fl in floors[limit:]:
                if (assignments[fl]['remaining_area'] >= info['area'] and
                    assignments[fl]['remaining_capacity'] >= info['cap']):
                    for b in info['blocks']:
                        assignments[fl]['assigned_blocks'].append(b)
                        assignments[fl]['remaining_area'] -= b['cumulative_area_sqm']
                        assignments[fl]['remaining_capacity'] -= b['max_occupancy']
                        assignments[fl]['assigned_departments'].add(b['department_sub_department'].strip())
                    placed = True
                    break
        # block-by-block fallback
        if not placed:
            for b in sorted(info['blocks'], key=lambda x: x['cumulative_area_sqm'], reverse=True):
                for fl in sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True):
                    if (assignments[fl]['remaining_area'] >= b['cumulative_area_sqm'] and
                        assignments[fl]['remaining_capacity'] >= b['max_occupancy']):
                        assignments[fl]['assigned_blocks'].append(b)
                        assignments[fl]['remaining_area'] -= b['cumulative_area_sqm']
                        assignments[fl]['remaining_capacity'] -= b['max_occupancy']
                        assignments[fl]['assigned_departments'].add(b['department_sub_department'].strip())
                        break
                else:
                    unassigned.append(b)

    # Phase 2: Typical – Neighborhood first
    nh_groups, rest = {}, []
    for blk in typ_blocks.to_dict('records'):
        nh = blk['neighborhood']
        if nh and nh != 'nan':
            nh_groups.setdefault(nh, []).append(blk)
        else:
            rest.append(blk)
    # assign neighborhood groups
    for group in nh_groups.values():
        area = sum(x['cumulative_area_sqm'] for x in group)
        cap = sum(x['max_occupancy'] for x in group)
        for fl in sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True):
            if (assignments[fl]['remaining_area'] >= area and
                assignments[fl]['remaining_capacity'] >= cap):
                for x in group:
                    assignments[fl]['assigned_blocks'].append(x)
                    assignments[fl]['remaining_area'] -= x['cumulative_area_sqm']
                    assignments[fl]['remaining_capacity'] -= x['max_occupancy']
                    assignments[fl]['assigned_departments'].add(x['department_sub_department'].strip())
                    cat = x['spacemix']
                    if cat == 'ME':       assignments[fl]['ME_area']       += x['cumulative_area_sqm']
                    elif cat == 'WE':     assignments[fl]['WE_area']       += x['cumulative_area_sqm']
                    elif cat == 'US':     assignments[fl]['US_area']       += x['cumulative_area_sqm']
                    elif cat.lower()=='support':    assignments[fl]['Support_area']    += x['cumulative_area_sqm']
                    elif cat.lower()=='speciality': assignments[fl]['Speciality_area'] += x['cumulative_area_sqm']
                break
        else:
            unassigned.extend(group)

    # Phase 2.2: Department unsplittable
    dept_groups, splittable = {}, []
    for blk in rest:
        key = blk['department_sub_department'].strip()
        if dept_splittable.get(key, -1) == -1:
            splittable.append(blk)
        else:
            dept_groups.setdefault(key, []).append(blk)
    # assign full dept groups
    for blks in dept_groups.values():
        area = sum(x['cumulative_area_sqm'] for x in blks)
        cap = sum(x['max_occupancy'] for x in blks)
        for fl in sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True):
            if (assignments[fl]['remaining_area'] >= area and
                assignments[fl]['remaining_capacity'] >= cap):
                for x in blks:
                    assignments[fl]['assigned_blocks'].append(x)
                    assignments[fl]['remaining_area'] -= x['cumulative_area_sqm']
                    assignments[fl]['remaining_capacity'] -= x['max_occupancy']
                    assignments[fl]['assigned_departments'].add(key)
                    cat = x['spacemix']
                    if cat == 'ME':       assignments[fl]['ME_area']       += x['cumulative_area_sqm']
                    elif cat == 'WE':     assignments[fl]['WE_area']       += x['cumulative_area_sqm']
                    elif cat == 'US':     assignments[fl]['US_area']       += x['cumulative_area_sqm']
                    elif cat.lower()=='support':    assignments[fl]['Support_area']    += x['cumulative_area_sqm']
                    elif cat.lower()=='speciality': assignments[fl]['Speciality_area'] += x['cumulative_area_sqm']
                break
        else:
            unassigned.extend(blks)

    # Phase 2.3: Splittable categories (ME first, then others)
    # ME
    me_blocks = [x for x in splittable if x['spacemix']=='ME']
    random.shuffle(me_blocks)
    for x in me_blocks:
        area, cap = x['cumulative_area_sqm'], x['max_occupancy']
        for fl in random.sample(floors, len(floors)):
            if assignments[fl]['remaining_area'] >= area:
                assignments[fl]['assigned_blocks'].append(x)
                assignments[fl]['remaining_area'] -= area
                assignments[fl]['remaining_capacity'] -= cap
                assignments[fl]['assigned_departments'].add(x['department_sub_department'].strip())
                assignments[fl]['ME_area'] += area
                break
        else:
            unassigned.append(x)
    # proportionally distribute others
    me_counts = {fl: sum(1 for b in assignments[fl]['assigned_blocks'] if b['spacemix']=='ME') for fl in floors}
    total_me = sum(me_counts.values())
    fract = {fl: (me_counts[fl]/total_me if total_me else 1/len(floors)) for fl in floors}
    categories = ['WE','US','Support','Speciality']
    for cat in categories:
        cat_blocks = [x for x in splittable if x['spacemix']==cat]
        n = len(cat_blocks)
        if n == 0:
            continue
        targets = {fl: int(round(fract[fl]*n)) for fl in floors}
        diff = n - sum(targets.values())
        # adjust rounding
        if diff > 0:
            # add to highest fractional parts
            fracs = {fl: fract[fl]*n - targets[fl] for fl in floors}
            for fl in sorted(fracs, key=fracs.get, reverse=True)[:diff]:
                targets[fl] += 1
        elif diff < 0:
            fracs = {fl: fract[fl]*n - targets[fl] for fl in floors}
            for fl in sorted(fracs, key=fracs.get)[: -diff]:
                targets[fl] -= 1
        random.shuffle(cat_blocks)
        counts = {fl: 0 for fl in floors}
        for x in cat_blocks:
            area, cap = x['cumulative_area_sqm'], x['max_occupancy']
            # choose floor with remaining target
            cands = [fl for fl in floors if counts[fl] < targets[fl]] or floors
            for fl in sorted(cands, key=lambda f: targets[f] - counts[f], reverse=True):
                if assignments[fl]['remaining_area'] >= area:
                    assignments[fl]['assigned_blocks'].append(x)
                    assignments[fl]['remaining_area'] -= area
                    assignments[fl]['remaining_capacity'] -= cap
                    assignments[fl]['assigned_departments'].add(x['department_sub_department'].strip())
                    if cat == 'WE': assignments[fl]['WE_area'] += area
                    elif cat == 'US': assignments[fl]['US_area'] += area
                    elif cat == 'Support': assignments[fl]['Support_area'] += area
                    elif cat == 'Speciality': assignments[fl]['Speciality_area'] += area
                    counts[fl] += 1
                    break
            else:
                unassigned.append(x)

    # Phase 3: Build output DataFrames
    # Detailed assignments
    detailed = []
    for fl, info in assignments.items():
        for b in info['assigned_blocks']:
            detailed.append({
                'Floor': fl,
                'Department': b['department_sub_department'],
                'Block_Name': b['block_name'],
                'Destination_Group': b['destination_group'],
                'SpaceMix': b['spacemix'],
                'Assigned_Area_SQM': b['cumulative_area_sqm'],
                'Max_Occupancy': b['max_occupancy']
            })
    detailed_df = pd.DataFrame(detailed)

    # Floor summary
    floor_summary_df = (
        detailed_df
        .groupby('Floor')
        .agg(
            Assgn_Blocks=('Block_Name','count'),
            Assgn_Area_SQM=('Assigned_Area_SQM','sum'),
            Total_Occupancy=('Max_Occupancy','sum')
        )
        .reset_index()
    )
    base = all_floor_data.rename(columns={
        'Name':'Floor',
        'Usable_Area_SQM':'Input_Usable_Area_SQM',
        'Max_Capacity':'Input_Max_Capacity'
    })[['Floor','Input_Usable_Area_SQM','Input_Max_Capacity']]
    floor_summary_df = base.merge(floor_summary_df, on='Floor', how='left').fillna(0)

        # Space mix by units
    cats = ['ME','WE','US','Support','Speciality']
    total_per_cat = {cat: len(typ_blocks[typ_blocks['spacemix']==cat]) for cat in cats}
    rows = []
    for fl in floors:
        cnts = {cat: sum(1 for b in assignments[fl]['assigned_blocks'] if b['spacemix']==cat) for cat in cats}
        for cat in cats:
            pct = (cnts[cat] / total_per_cat[cat] * 100) if total_per_cat[cat] else 0
            rows.append({'Floor': fl, 'SpaceMix': cat, '%spaceMix': round(pct, 2)})
    space_mix_df = pd.DataFrame(rows)

    # Unassigned blocks
    unassigned_df = pd.DataFrame([
        {
            'Department': b['department_sub_department'],
            'Block_Name': b['block_name'],
            'Destination_Group': b['destination_group'],
            'SpaceMix': b['spacemix'],
            'Area_SQM': b['cumulative_area_sqm'],
            'Max_Occupancy': b['max_occupancy']
        } for b in unassigned
    ])

    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 4: Generate & Export all 3 plans
# ----------------------------------------
plans = [
    ('centralized','stack_plan_centralized28.xlsx'),
    ('semi','stack_plan_semi_centralized28.xlsx'),
    ('decentralized','stack_plan_decentralized28.xlsx')
]
for mode, fname in plans:
    det, fs, sm, un = run_stack_plan(mode)
    with pd.ExcelWriter(fname) as writer:
        det.to_excel(writer, sheet_name='Detailed', index=False)
        fs.to_excel(writer, sheet_name='Floor_Summary', index=False)
        sm.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
        un.to_excel(writer, sheet_name='Unassigned', index=False)
    print(f"✅ Generated {fname}")

KeyError: 'Department_Sub-Department'

# DR

In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load Input Sheets & Normalize
# ----------------------------------------

excel_path = '/content/BA- R1.xlsx'  # ← adjust if needed

# 1.1 Floors sheet
all_floor_data = pd.read_excel(
    excel_path,
    sheet_name='Program Table Input 2 - Floor'
)
all_floor_data.columns = all_floor_data.columns.str.strip()

# Normalize usable-area & capacity column names
floor_col_map = {}
for c in all_floor_data.columns:
    key = c.lower().replace(' ', '').replace('_','')
    if 'usable' in key and 'area' in key:
        floor_col_map[c] = 'Usable_Area'
    elif 'capacity' in key or 'loading' in key:
        floor_col_map[c] = 'Max_Assignable_Floor_loading_Capacity'
all_floor_data = all_floor_data.rename(columns=floor_col_map)

# 1.2 Blocks sheet
all_block_data = pd.read_excel(
    excel_path,
    sheet_name='Existing Program Table Input 1.'
)
all_block_data.columns = all_block_data.columns.str.strip()

# 1.3 Department Split sheet
department_split_data = pd.read_excel(
    excel_path,
    sheet_name='Department Split',
    skiprows=1
)
department_split_data.columns = department_split_data.columns.str.strip()
department_split_data = department_split_data.rename(
    columns={'BU_Department_Sub-Department': 'Department_Sub-Department'}
)

# 1.4 Adjacency sheet
xls = pd.ExcelFile(excel_path)
adjacency_sheet_name = [n for n in xls.sheet_names if "Adjacency" in n][0]
raw_adj = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_adj.apply(pd.to_numeric, errors='coerce')
adjacency_data.index = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()

# 1.5 De-Centralized Logic sheet
logic_df = pd.read_excel(
    excel_path,
    sheet_name='De-Centralized Logic',
    header=None
)
De_Centralized_data = {}
current = None
for _, r in logic_df.iterrows():
    cell = str(r[0]).strip() if pd.notna(r[0]) else ""
    if cell in ["Centralised", "Semi Centralized", "DeCentralised"]:
        current = cell
        De_Centralized_data[current] = {"Add": 0}
    elif current and "Add into" in cell:
        De_Centralized_data[current]["Add"] = int(r[1]) if pd.notna(r[1]) else 0
for k in ["Centralised", "Semi Centralized", "DeCentralised"]:
    De_Centralized_data.setdefault(k, {"Add": 0})

# ----------------------------------------
# Step 2: Preprocess Blocks
# ----------------------------------------

# Split by asset type
immovable_blocks = all_block_data[all_block_data['Immovable-Movable Asset'].str.strip() == 'Immovable Asset']
movable_blocks   = all_block_data[all_block_data['Immovable-Movable Asset'].str.strip() != 'Immovable Asset']

destination_blocks = movable_blocks[movable_blocks['Typical_Destination'].isin(['Destination','both'])]
typical_blocks     = movable_blocks[movable_blocks['Typical_Destination']=='Typical']

# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

# list of all floors
floors = list(initialize_floor_assignments(all_floor_data).keys())

# ----------------------------------------
# Step 4: Core Assignment Function
# ----------------------------------------

def run_stack_plan(mode):
    # initialize per-run structures
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []

    # helper to map short names
    import re
    def clean_floor_name(f): return re.sub(r'^L\d{3}', '', f).strip()
    floor_name_map = {clean_floor_name(r['Name']): r['Name'].strip() for _,r in all_floor_data.iterrows()}

    # 4.1 Assign immovable blocks by level
    for _, blk in immovable_blocks.iterrows():
        raw = str(blk['Level']).strip()
        fl = floor_name_map.get(raw)
        if fl and assignments[fl]['remaining_area']>=blk['Cumulative_Block_Circulation_Area_(SQM)']:
            assignments[fl]['assigned_blocks'].append(blk.to_dict())
            assignments[fl]['assigned_departments'].add(blk['Department_Sub-Department'])
            assignments[fl]['remaining_area'] -= blk['Cumulative_Block_Circulation_Area_(SQM)']
        else:
            unassigned_blocks.append(blk.to_dict())

    # 4.2 Determine destination floor count
    def dest_count():
        if mode=='centralized': return 2
        key = 'Semi Centralized' if mode=='semi' else 'DeCentralised'
        return 2 + De_Centralized_data.get(key,{}).get('Add',0)
    max_dest = min(dest_count(), len(floors))

    # assign destination groups
    dest_groups = {}
    for _,blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']; dest_groups.setdefault(grp, {'blocks':[]}).update({'blocks': dest_groups.get(grp,{}).get('blocks',[])+[blk.to_dict()]})

    for grp,info in dest_groups.items():
        # try whole
        placed=False
        area=sum(b['Cumulative_Block_Circulation_Area_(SQM)'] for b in info['blocks'])
        for fl in floors[:max_dest]:
            if assignments[fl]['remaining_area']>=area:
                for b in info['blocks']:
                    assignments[fl]['assigned_blocks'].append(b)
                    assignments[fl]['assigned_departments'].add(b['Department_Sub-Department'])
                assignments[fl]['remaining_area']-=area; placed=True; break
        if not placed:
            for b in info['blocks']:
                placed=False
                for fl in sorted(floors, key=lambda f:assignments[f]['remaining_area'], reverse=True):
                    if assignments[fl]['remaining_area']>=b['Cumulative_Block_Circulation_Area_(SQM)']:
                        assignments[fl]['assigned_blocks'].append(b)
                        assignments[fl]['assigned_departments'].add(b['Department_Sub-Department'])
                        assignments[fl]['remaining_area']-=b['Cumulative_Block_Circulation_Area_(SQM)']
                        placed=True; break
                if not placed: unassigned_blocks.append(b)

    # 4.3 Typical blocks: just place until full
    for _,blk in typical_blocks.iterrows():
        placed=False
        for fl in sorted(floors, key=lambda f:assignments[f]['remaining_area'], reverse=True):
            if assignments[fl]['remaining_area']>=blk['Cumulative_Block_Circulation_Area_(SQM)']:
                assignments[fl]['assigned_blocks'].append(blk.to_dict())
                assignments[fl]['assigned_departments'].add(blk['Department_Sub-Department'])
                assignments[fl]['remaining_area']-=blk['Cumulative_Block_Circulation_Area_(SQM)']
                placed=True; break
        if not placed: unassigned_blocks.append(blk.to_dict())

    # Phase 5: Build outputs
    detailed=[]
    for fl,info in assignments.items():
        for b in info['assigned_blocks']:
            detailed.append({
                'Block_ID': b.get('Block_ID'),
                'Floor': fl,
                'Department': b.get('Department_Sub-Department'),
                'Block_Name': b.get('Block_Name'),
                'Destination_Group': b.get('Destination_Group'),
                'SpaceMix': b.get('SpaceMix_(ME_WE_US_Support_Speciality)'),
                'Assigned_Area_SQM': b.get('Cumulative_Block_Circulation_Area_(SQM)'),
                'Max_Occupancy': b.get('Max_Occupancy_with_Capacity'),
                'Asset_Type': b.get('Immovable-Movable Asset')
            })
    detailed_df = pd.DataFrame(detailed)

    floor_sum = (detailed_df.groupby('Floor')
                 .agg(Assgn_Blocks=('Block_Name','count'),
                      Assgn_Area_SQM=('Assigned_Area_SQM','sum'))
                 .reset_index())

    space_rows=[]
    cats=['ME','WE','US','Support','Speciality']
    totals={c:len(detailed_df[detailed_df['SpaceMix']==c]) for c in cats}
    for fl in floors:
        df_fl=detailed_df[detailed_df['Floor']==fl]
        for c in cats:
            cnt=len(df_fl[df_fl['SpaceMix']==c]); tot=totals[c] or 1
            pct_fl=cnt/len(df_fl)*100 if len(df_fl) else 0
            pct_ov=cnt/tot*100
            space_rows.append({'Floor':fl,'SpaceMix':c,'Unit_Count_on_Floor':cnt,
                               'Pct_of_Floor_UC':round(pct_fl,2),'Pct_of_Overall_UC':round(pct_ov,2)})
    space_df=pd.DataFrame(space_rows)

    unass=[{'Department':b.get('Department_Sub-Department'),
            'Block_Name':b.get('Block_Name'),
            'Destination_Group':b.get('Destination_Group'),
            'SpaceMix':b.get('SpaceMix_(ME_WE_US_Support_Speciality)'),
            'Area_SQM':b.get('Cumulative_Block_Circulation_Area_(SQM)'),
            'Max_Occupancy':b.get('Max_Occupancy_with_Capacity'),
            'Asset_Type':b.get('Immovable-Movable Asset')} for b in unassigned_blocks]
    unassigned_df=pd.DataFrame(unass)

    return detailed_df,floor_sum,space_df,unassigned_df

# ----------------------------------------
# Step 6: Run & Export
# ----------------------------------------
central, cen_sum, cen_space, cen_un = run_stack_plan('centralized')
semi, sem_sum, sem_space, sem_un = run_stack_plan('semi')
dec, dec_sum, dec_space, dec_un = run_stack_plan('decentralized')

with pd.ExcelWriter('stack_plan_outputs.xlsx') as w:
    central.to_excel(w,'Central_Detailed',index=False)
    cen_sum.to_excel(w,'Central_Summary',index=False)
    cen_space.to_excel(w,'Central_SpaceMix',index=False)
    cen_un.to_excel(w,'Central_Unassigned',index=False)
    semi.to_excel(w,'Semi_Detailed',index=False)
    sem_sum.to_excel(w,'Semi_Summary',index=False)
    sem_space.to_excel(w,'Semi_SpaceMix',index=False)
    sem_un.to_excel(w,'Semi_Unassigned',index=False)
    dec.to_excel(w,'Dec_Detailed',index=False)
    dec_sum.to_excel(w,'Dec_Summary',index=False)
    dec_space.to_excel(w,'Dec_SpaceMix',index=False)
    dec_un.to_excel(w,'Dec_Unassigned',index=False)

print("✔ Code executed: three stack plan outputs generated.")